# Import bibliotek

In [ ]:
# --- Przetwarzanie danych ---
import numpy as np
import pandas as pd

# --- Wizualizacja ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Preprocessing ---
from scipy.signal import correlate2d
from scipy.stats import bartlett, levene, fligner
from skimage.metrics import structural_similarity, variation_of_information, normalized_root_mse
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.metrics import (
    silhouette_score,
    silhouette_samples,
    homogeneity_completeness_v_measure,
    adjusted_rand_score,
    adjusted_mutual_info_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    fowlkes_mallows_score,
    mean_squared_error,
)
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
import umap

# --- Modelowanie ---
import optuna
import optunahub
from sklearn.cluster import FeatureAgglomeration, KMeans
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import (
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    VotingClassifier,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# --- Ewaluacja ---
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    make_scorer,
)
from sklearn.model_selection import cross_val_score

# --- Przetwarzanie obrazów ---
import cv2 as cv2
from skimage.color import rgb2gray
from skimage.feature import hog, graycomatrix, graycoprops
from skimage.filters import gabor_kernel, gabor
from skimage.measure import label, regionprops
from skimage import img_as_float32
from skimage.transform import resize

# --- Zrównoleglanie ---
from joblib import Parallel, delayed
from multiprocessing import Pool

# --- Inne ---
import time
import gc
import warnings
import sqlite3

# --- Sklearn ---
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

# --- Skimage ---
from skimage.transform import AffineTransform, warp
from skimage.filters import gaussian, unsharp_mask
from skimage.measure import find_contours
from skimage.morphology import erosion, closing, opening,  disk

# --- Dodatkowe importy ---
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import FunctionTransformer
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "12" 

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    auc,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)
from sklearn.base import BaseEstimator
import warnings
from typing import List, Dict, Optional, Any, Tuple, Union, Literal, Callable

try:
    import optunahub
    OPTUNAHUB_AVAILABLE = True
except ImportError:
    OPTUNAHUB_AVAILABLE = False
    print("Ostrzeżenie: Moduł optunahub nie jest zainstalowany. Używam domyślnego TPESampler.")
import warnings
import traceback
import gc
from typing import List, Dict, Optional, Any, Tuple, Union, Literal, Callable
from tqdm.auto import tqdm


from skimage.feature import graycomatrix, graycoprops
from esda.moran import Moran
from libpysal.weights import lat2W
from libpysal.weights import KNN
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
import statsmodels.api as sm
from joblib import Parallel, delayed


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from IPython.display import display, Markdown



# EDA

In [ ]:
train = pd.read_csv('mnist_train.csv')
test  = pd.read_csv('mnist_test.csv')
train.info()
test.info()

# Podział na cechy i etykiety
X_train = train.drop('label', axis=1).values.reshape(-1, 28, 28)
y_train = train['label'].values

X_test = test.drop('label', axis=1).values.reshape(-1, 28, 28)
y_test = test['label'].values

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

 ### Wizualizacja podstawowych cech

In [ ]:
def visualize_statistics(X, label):
    """Oblicza i wizualizuje statystyki dla danej klasy."""

    # Obliczenie statystyk
    mean_pixels = np.mean(X, axis=0)
    std_pixels = np.std(X, axis=0)
    median_pixels = np.median(X, axis=0)
    q25_pixels = np.quantile(X, 0.25, axis=0)
    q75_pixels = np.quantile(X, 0.75, axis=0)
    min_pixels = np.mean(np.percentile(X, [0, 2.5], axis=0), axis=0)
    max_pixels = np.mean(np.percentile(X, [97.5, 100], axis=0), axis=0)

    # Wizualizacja wszystkich statysty
    _, axes = plt.subplots(2, 4, figsize=(15, 8))

    sns.heatmap(mean_pixels, cmap='gray', ax=axes[0, 0])
    axes[0, 0].set_title(f"Średnia")

    sns.heatmap(std_pixels, cmap='viridis', ax=axes[0, 1])
    axes[0, 1].set_title(f"Odchylenie std.")

    axes[0, 2].imshow(median_pixels, cmap='gray')
    axes[0, 2].set_title("Mediana")
    axes[0, 2].axis('off')

    axes[0, 3].imshow(q25_pixels, cmap='gray')
    axes[0, 3].set_title("Kwartyl 25%")
    axes[0, 3].axis('off')

    axes[1, 0].imshow(q75_pixels, cmap='gray')
    axes[1, 0].set_title("Kwartyl 75%")
    axes[1, 0].axis('off')

    axes[1, 1].imshow(min_pixels, cmap='gray')
    axes[1, 1].set_title("2.5% minimum")
    axes[1, 1].axis('off')

    axes[1, 2].imshow(max_pixels, cmap='gray')
    axes[1, 2].set_title("2.5% maksimum")
    axes[1, 2].axis('off')

    axes[1, 3].hist(X.flatten(), bins=15)
    axes[1, 3].set_title(f"Histogram\nwartości pikseli")
    axes[1, 3].set_xlabel('Wartość piksela')
    axes[1, 3].set_ylabel('Liczba wystąpień')

    plt.suptitle(f"Etykieta: {label}", fontsize=14)
    plt.tight_layout()
    plt.show()

# Grupowanie po etykietach w zbiorze treningowym
for label in range(10):
    X_train_group = X_train[y_train == label]
    X_test_group = X_test[y_test == label]

    # Wizualizacja statystyk dla zbioru treningowego
    print("Zbiór treningowy:")
    visualize_statistics(X_train_group, label)

    # Wizualizacja statystyk dla zbioru testowego
    print("\nZbiór testowy:")
    visualize_statistics(X_test_group, label)

In [ ]:
def porownaj_srednie_obrazow(X_train, y_train, X_test, y_test):
    """
    Porównuje średnie obrazy dla każdej klasy w zbiorze treningowym i testowym 
    za pomocą trzech metryk: SSIM, NRMSE i Variation of Information.

    Args:
        X_train: Zbiór treningowy obrazów (numpy array).
        y_train: Etykiety dla zbioru treningowego (numpy array).
        X_test: Zbiór testowy obrazów (numpy array).
        y_test: Etykiety dla zbioru testowego (numpy array).

    Returns:
        DataFrame zawierający wyniki dla każdej metryki i klasy.
    """

    wyniki = {}
    for label in range(10):
        X_train_group = X_train[y_train == label]
        X_test_group = X_test[y_test == label]

        # Obliczenie średnich obrazów
        sredni_obraz_train = np.mean(X_train_group, axis=0)
        sredni_obraz_test = np.mean(X_test_group, axis=0)

        # Obliczenie metryk
        ssim = structural_similarity(sredni_obraz_train, sredni_obraz_test, data_range=sredni_obraz_train.max() - sredni_obraz_train.min())
        nrmse = normalized_root_mse(sredni_obraz_train, sredni_obraz_test)

        # Variation of Information
        vi = variation_of_information(sredni_obraz_train.astype(int), sredni_obraz_test.astype(int)) 

        wyniki[label] = {
            'ssim': ssim,
            'nrmse': nrmse,
            'vi': vi[0],  
        }

    return pd.DataFrame(wyniki).transpose()  # Utwórz DataFrame i transponuj go

wyniki_df = porownaj_srednie_obrazow(X_train, y_train, X_test, y_test)
display(wyniki_df)


plt.figure(figsize=(14, 12))

# SSIM
plt.subplot(1, 3, 1)
sns.barplot(x=wyniki_df.index, y=wyniki_df['ssim'], hue=wyniki_df.index, palette="viridis", legend=False)
plt.title('SSIM')
plt.xlabel('Klasa')
plt.ylabel('Wartość SSIM')
plt.ylim(0.97, 1)
for i, v in enumerate(wyniki_df['ssim']):
    plt.text(i, v, f"{v:.2f}", ha="center", va="bottom")

# NRMSE
plt.subplot(1, 3, 2)
sns.barplot(x=wyniki_df.index, y=wyniki_df['nrmse'], hue=wyniki_df.index, palette="viridis", legend=False)
plt.title('NRMSE')
plt.xlabel('Klasa')
plt.ylabel('Wartość NRMSE')

for i, v in enumerate(wyniki_df['nrmse']):
    plt.text(i, v, f"{v:.2f}", ha="center", va="bottom")

# VI
plt.subplot(1, 3, 3)
sns.barplot(x=wyniki_df.index, y=wyniki_df['vi'], hue=wyniki_df.index, palette="viridis", legend=False)
plt.title('Variation of Information')
plt.xlabel('Klasa')
plt.ylabel('Wartość VI')

for i, v in enumerate(wyniki_df['vi']):
    plt.text(i, v, f"{v:.2f}", ha="center", va="bottom")

plt.tight_layout()
plt.show()

#### Podsumowanie

Powyżej przedstawiono analizę podstawowych własności zbioru danych MINST, skupiając się na zestawieniu średnich obrazów należących do poszczególnych cyfr (etykiet) w zbiorach treningowym i testowym. \
Na początek wizualizacja średnich wartości obrazów, dla etykiet będących w danych zbiorach. Przedstawiono statystyki opisujące wartość średnią, odchylenie standardowem medianę, kwartyle (25% i 75%), 2.5% minimum, 2.5% maksimum oraz histogram wartości pikseli, dla lokaliacji przestrzennej danego piksela. \ 
W kolejnym etapie podobie dla średnich obrazów obliczono statystyki pomiędzędzy zbiorem treningowy a testowym: 
* SSIM, który mierzy strukturalne podobieństwo między dwoma (średnimi/wypadkowymi) obrazami. Wartość 1 oznacza idealne dopasowanie. 
* NRMSE, czyli błąd średniokwadratowy. Niższe wartości NRMSE wskazują na mniejszy błąd i większe podobieństwo. 
* VI, który informuje o ilości informacji utraconej lub uzyskanej. Niższe wartości VI wskazują na większe podobieństwo informacyjne.

Wizualna anliza statystyk dla obrazów nie przedstawia znaczących różnic po między zbiorami. Dostrzec można różnicę w obrazach gdzie wykreślone jest odchylenie standardowe. Największe różnice widoczne są w przypadku 2.5% kwantyla z wartości maksymalnych. Większa przestrzeń pikseli o maksymalnej jasnośości wystepuje w zbiorze treningowym względem zbioru "testowego". Można przypuszczać, żę wynika to z tego ze w związku z większą liczbą elementów w zbiorze treningowym, można spotkać więcej etykiet które znajdują się na skrajach granic ograniczjąych. \
W przypadku statystyk dla średnich obrazów. Warttości wszystkich etykiet obliczonych dla SSIM są bardzo wysokie, co wyrażnie sugeruje duże struktualne podobieństwo pomiędzy średninimi obrazami. W przypadku VI możemy zauważyć coś na pierszy, rzut dość nie prawdopodobnego, mianowicie wartość dla cyfry 1, wynosi około 0.41, podczas gdy dla pozostały jest w zakresie około 0.8-0.95. Należy mieć jednak na uwadze, że nie wskazuje to znacznie większego podobiebieństwa wzklędem pozostałych etykiet. Za to na mniejszą ilości informacji potrzebnych na opisanie względnych róznic w średnim obrazie pomiędzy dwoma zbiorami. Można przypuszczać, że liczba "1" z względu na swój kształtm, który praktycznie jest pionową linią, wymga mniejszje reprezentacji przestrzennej, a tym samym może to przekłądać się na mniejszą wariację przestrzenną pomiędzy zbiorami. W związku z czym VI należy interpretować w kontekście waiancji danych. Natomiast z względu na uzupełnienie tego metrykami SSIM i NRMSE, daje to inny, bardziej związany z teorią informacji wgląd na podobieństwo między zbiorami treningowym i testowym. \ 
\
Analiza wykazała duże podobieństwo między zbiorem treningowym i testowym MNIST pod względem średniego rozkładu przestrzennego wartości pikseli dla poszczególnych cyfr. Zarówno wizualizacje statystyk opisowych, jak i metryki porównujące średnie obrazy (SSIM, NRMSE, VI) potwierdzają tę obserwację. Wskazuje to na wysokie podobieństwo między zbiorem treningowym, a testowym i sugeruje że zbiór testowy jest reprezentatywny dla zbioru treningowego. Więc modele uczenia maszynowego wytrenowane na zbiorze treningowym powinny dobrze generalizować na nowe, niewidziane dane ze zbioru testowego. 

### Analiza miar zależności przestrzennych, oraz tekstur

In [ ]:
def calculate_glcm_features(image, distances=[1], angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]):
    """Oblicza cechy tekstury z macierzy GLCM dla danego obrazu."""

    glcm = graycomatrix(
        image,
        distances=distances,
        angles=angles,
        levels=256,
        symmetric=True,
        normed=True,
    )

    # Obliczenie wszystkich cech GLCM jednocześnie
    features = {}
    props = ["contrast", "dissimilarity", "homogeneity", "energy", "correlation", "ASM"]
    for prop in props:
      features[prop] = graycoprops(glcm, prop)[0, 0]

    # Dodanie wariancji i entropii
    features["variance"] = np.var(image)
    hist = np.histogram(image, bins=256, range=(0, 256))[0]
    features["entropy"] = entropy(hist)
    return features


def calculate_spatial_autocorrelation(image, nlags=10):
    """Oblicza średnią i odchylenie standardowe autokorelacji przestrzennej."""
    autocorr = sm.tsa.acf(image.flatten(), nlags=min(nlags, image.size - 1), fft=True)
    return np.mean(autocorr), np.std(autocorr)


def calculate_moran_index(image):
    """Oblicza indeks Morana dla danego obrazu."""
    w = lat2W(*image.shape)
    moran_i = Moran(image.flatten(), w)
    return moran_i.I


def calculate_spatial_features(images, n_jobs=-1):
    """
    Oblicza miary zależności przestrzennych i cechy tekstur dla obrazów.

    Args:
        images: Tablica obrazów w skali szarości.
        n_jobs: Liczba rdzeni do użycia w Parallel. -1 oznacza użycie wszystkich dostępnych rdzeni.

    Returns:
        DataFrame: DataFrame z obliczonymi cechami.
    """

    def _calculate_features(image_id, image):
        """Funkcja wewnętrzna do obliczania cech dla pojedynczego obrazu."""

        # Konwersja obrazu do typu uint8 w zakresie 0-255
        if image.dtype != np.uint8:
            image = (image * 255).astype(np.uint8)

        # Miary zależności przestrzennych
        autocorr_mean, autocorr_std = calculate_spatial_autocorrelation(image)
        moran_i_mean = calculate_moran_index(image)

        # Analiza tekstur
        glcm_features = calculate_glcm_features(image)

        return {
            "image_id": image_id,
            "autocorr_mean": autocorr_mean,
            "autocorr_std": autocorr_std,
            "moran_I": moran_i_mean,
            **glcm_features,
        }

    features = Parallel(n_jobs=n_jobs)(
        delayed(_calculate_features)(image_id, image)
        for image_id, image in enumerate(images)
    )

    return pd.DataFrame(features)


# 1. Obliczenia dla całego zbioru treningowego
features_original = calculate_spatial_features(X_train)
features_original['label'] = y_train

# 2. Statystyki opisowe dla całego zbioru
print("Statystyki dla całego zbioru:")
display(features_original.drop(columns=['image_id', 'label']).describe())

# 3. Statystyki opisowe i wykresy dla poszczególnych cyfr
features_by_label = []
for label in range(10):
    features_label = features_original[features_original['label'] == label].copy()
    features_by_label.append(features_label)
    print(f"\nStatystyki dla cyfry {label}:")
    display(features_label.drop(columns=['image_id', 'label']).describe())

# 4. Połączenie danych dla wizualizacji
combined_features = pd.concat(features_by_label, ignore_index=True)

# 5. Wizualizacja
features_to_plot = ["moran_I", "autocorr_mean", "autocorr_std", "contrast", "dissimilarity", "homogeneity", "energy", "correlation", "ASM", "variance", "entropy"]

# Wykresy tylko dla poszczególnych cyfr (bez całego zbioru)
for feature in features_to_plot:
    plt.figure(figsize=(14, 12))
    sns.violinplot(x='label', y=feature, data=combined_features)
    plt.title(f"Rozkład cechy '{feature}' dla różnych cyfr")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

### Wizualizacja wartości i wektrów własnych

In [ ]:
def analyze_eigen(X, label):
    """Analizuje i wizualizuje wartości i wektory własne dla danej klasy,
    wraz ze skumulowaną wariancją."""

    # Spłaszczenie danych
    X_flat = X.reshape(X.shape[0], -1)

    # Obliczenie macierzy kowariancji
    cov_matrix = np.cov(X_flat, rowvar=False)

    # Obliczenie wartości i wektorów własnych
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sortowanie wartości własnych
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Obliczenie skumulowanej wariancji
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    num_eigenvectors_to_display = 9
    
    # Siatka dla wykresów
    fig = plt.figure(figsize=(24, 12))
    gs = fig.add_gridspec(3, num_eigenvectors_to_display, height_ratios=[2, 2, 1])

    # Wykres wartości własnych
    ax1 = fig.add_subplot(gs[0, :])
    color = 'tab:blue'
    ax1.set_xlabel('Numer wartości własnej')
    ax1.set_ylabel('Wartość własna', color=color)
    ax1.plot(eigenvalues, marker='o', color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True)
    ax1.set_title(f"Wartości własne, wektory własne i skumulowana wariancja dla etykiety {label}")

    # Utworzenie drugiej osi Y dla powiększenia
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Wartość własna (powiększenie)', color=color)
    ax2.plot(range(200, len(eigenvalues)), eigenvalues[200:], marker='o', color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    # Wykres skumulowanej wariancji
    ax3 = fig.add_subplot(gs[1, :])
    color = 'tab:green'
    ax3.set_xlabel('Liczba składowych')
    ax3.set_ylabel('Skumulowana wariancja', color=color)
    ax3.plot(cumulative_variance, marker='o', color=color)
    ax3.tick_params(axis='y', labelcolor=color)
    ax3.grid(True)

    # Dodanie linii poziomej na wykresie skumulowanej wariancji dla 95%
    ax3.axhline(y=0.98, color='r', linestyle='--')
    ax3.text(0, 0.98, '98%', color='r', va='bottom', ha='left')

    # Wizualizacja wektorów własnych
    for i in range(num_eigenvectors_to_display):
        ax = fig.add_subplot(gs[2, i])
        ax.imshow(eigenvectors[:, i].reshape(28, 28), cmap='gray')
        ax.set_title(f"Wektor własny {i+1}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Iteracja po etykietach
for label in range(10):
    X_train_group = X_train[y_train == label]
    analyze_eigen(X_train_group, label)

#### Podsumowanie

Przejdźmy do analizy wartości i wektorów własnych, obliczonych dla każdej cyfry (etykiety) w zbiorze treningowym MNIST. Wartości własne informują nas o 'znaczeniu' odpowiadających im składowych głównych (wektorów własnych) i pozwalają określić, ile z nich jest potrzebnych do opisania większości wariancji danych. Innymi słowy, większa wartość własna oznacza, że odpowiadająca jej składowa główna wyjaśnia większą część zmienności w danych. Z definicji, wektor własny to taki wektor, który po przemnożeniu przez macierz zachowuje swój kierunek, zmieniając co najwyżej swoją długość. 

Wektory własne macierzy kowariancji, ianczej składowe główne pozwalają wskazać kierunki największej wariancji w przestrzeni danych 784-wymiarowej (28x28 pikseli) obrazów z boiru MINST. Wartość własna informuje jak duża jest natomiast wariancja danych wzdłuż odpowiadającego jej wektora własnego, czyli składowej głównej. Po transformacji danych do przestrzeni wyznaczonej przez wektory własne, współrzędne wektora wzdłuż odpowiadającego mu kierunku są przemnożone przez wartość własną. 

Analizując wykres wartości własny możemy zoabczyć, że znacząca część wariancji danych jest wyjaśniona przez pierwsze 100 składowych głównych. Dodatkowo, wszystkie etykiety poza "1" potrzebują około 500 wartości własnych (czerwona linia na wykresie), a etykieta "1" 400 do wyjaśnienia większości wariancji co potwierdza wcześniejsze przypuszczenia. 


### Metody składowych głównych

In [ ]:
def reduce_dimensionality_to_k(X_train, y_train, method_name, n_components):
    """
    Redukuje wymiarowość danych MNIST za pomocą określonej metody do k wymiarów.

    Args:
        X_train: Dane treningowe MNIST.
        y_train: Etykiety danych treningowych.
        method_name: Nazwa metody redukcji wymiarów.
        n_components: Liczba wymiarów, do których należy zredukować dane.

    Returns:
        Dane po redukcji wymiarów, czas trwania operacji.
    """

    X_train_flat = X_train.reshape(X_train.shape[0], -1)

    methods = {
        "PCA": PCA,
        "KernelPCA": KernelPCA,
        "t-SNE": TSNE,
        "UMAP": umap.UMAP,
        "LLE": LocallyLinearEmbedding,
        "LDA": LinearDiscriminantAnalysis,
    }

    if method_name not in methods:
        raise ValueError(f"Nieznana metoda: {method_name}")

    # Ustawienie parametrów dla każdej metody
    method_params = {
        "PCA": {"n_components": n_components, "random_state": 42},
        "KernelPCA": {"n_components": n_components, "kernel": "cosine", "random_state": 42},
        "t-SNE": {"n_components": n_components, "random_state": 42, "perplexity": 30},
        "UMAP": {"n_components": n_components, "random_state": 42, "n_neighbors": 42},
        "LLE": {"n_components": n_components, "random_state": 42, "n_neighbors": 42,  'method': 'ltsa', "eigen_solver": "dense"},
        "LDA": {"n_components": 3},
    }

    method = methods[method_name](**method_params[method_name])

    start_time = time.time()
    if method_name == "LDA":
        X_transformed = method.fit_transform(X_train_flat, y_train)
    else:
        X_transformed = method.fit_transform(X_train_flat)
    end_time = time.time()

    return X_transformed, end_time - start_time


def cluster_and_evaluate(X, y_true, clustering_method, method_name="Raw Data", skip_silhouette=False):
    """
    Przeprowadza klasteryzację, ocenia jej jakość i zwraca wyniki w formie słownika.

    Args:
        X: Dane (surowe lub po redukcji).
        y_true: Prawdziwe etykiety.
        clustering_method: Nazwa metody klasteryzacji.
        method_name: Nazwa metody redukcji wymiarów (domyślnie "Raw Data").
        skip_silhouette: Czy pominąć obliczanie silhouette_score (domyślnie False).

    Returns:
        Słownik z wynikami.
    """

    X_flat = X.reshape(X.shape[0], -1)

    clusterers = {
        "KMeans": KMeans(n_clusters=10, random_state=42, n_init="auto"),
        "GMM": GaussianMixture(n_components=10, random_state=42),
        "Agglomerative": AgglomerativeClustering(n_clusters=10),
        "DBSCAN": DBSCAN(eps=0.5, min_samples=5), 
        "Ward": AgglomerativeClustering(n_clusters=10, linkage='ward')
    }

    if clustering_method not in clusterers:
        raise ValueError(f"Nieznana metoda klasteryzacji: {clustering_method}")
    
    # Obsługa przypadku, gdy danych jest zbyt mało dla OPTICS lub DBSCAN
    if (clustering_method == "OPTICS" or clustering_method == "DBSCAN") and X_flat.shape[0] < 10:
        print(f"Warning: Zbyt mało danych dla {clustering_method} po redukcji {method_name}. Pomijanie {clustering_method}.")
        return {
            "Reduction Method": method_name,
            "Clustering Method": clustering_method,
            "Silhouette Score (Clustering)": None,
            "Adjusted Rand Score": None,
            "Adjusted Mutual Information": None,
            "Fowlkes-Mallows Score": None,
            "Homogeneity": None,
            "Completeness": None,
            "V-Measure": None,
            "Clustering Time (s)": 0,
            "Reduction Time (s)": 0 if method_name == "Raw Data" else None,
            "y_pred": None,
        }

    clusterer = clusterers[clustering_method]

    start_time = time.time()
    
    y_pred = clusterer.fit_predict(X_flat)

    end_time = time.time()
    time_taken = end_time - start_time

    n_labels = len(np.unique(y_pred))

    ars = adjusted_rand_score(y_true, y_pred)
    amis = adjusted_mutual_info_score(y_true, y_pred)
    fms = fowlkes_mallows_score(y_true, y_pred)
    homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(y_true, y_pred)

    if n_labels <= 1:
        print(f"Warning: {clustering_method} found only one cluster. Silhouette score set to None.")
        silhouette = None
    elif (method_name == "Raw Data" and skip_silhouette) or (method_name == "Raw Data" and not skip_silhouette):
        print("Pominięto obliczanie silhouette_score dla surowych danych.")
        silhouette = None
    elif method_name != "Raw Data":
        # Użyto mniejszej podpróbki (5000) dla silhouette_score
        if X_flat.shape[0] > 5000:
            sample_size = 10000
            indices = np.random.choice(X_flat.shape[0], sample_size, replace=False)
            X_sample = X_flat[indices]
            y_sample = y_pred[indices]
            silhouette = silhouette_score(X_sample, y_sample)
        else:
            silhouette = silhouette_score(X_flat, y_pred)
    else:
        silhouette = None

    return {
        "Reduction Method": method_name,
        "Clustering Method": clustering_method,
        "Silhouette Score (Clustering)": silhouette,
        "Adjusted Rand Score": ars,
        "Adjusted Mutual Information": amis,
        "Fowlkes-Mallows Score": fms,
        "Homogeneity": homogeneity,
        "Completeness": completeness,
        "V-Measure": v_measure,
        "Clustering Time (s)": time_taken,
        "Reduction Time (s)": 0 if method_name == "Raw Data" else None,
        "y_pred": y_pred,
    }


method_names = ["PCA", "KernelPCA", "LLE", "LDA", "t-SNE", "UMAP"]
clustering_methods = ["KMeans", "GMM"]
results = []

# Opcje
n_components = 3 
cluster_on_raw_data = False
skip_silhouette_on_raw = False

# Klasteryzacja na surowych danych
if cluster_on_raw_data:
    print("\n--- Klasteryzacja na surowych danych ---\n")
    for clustering_method in clustering_methods:
        print(f"Rozpoczynam klasteryzację: {clustering_method}")
        result = cluster_and_evaluate(
            X_train, y_train, clustering_method, "Raw Data", skip_silhouette_on_raw
        )
        print(f"Zakończono klasteryzację. Czas: {result['Clustering Time (s)']:.2f}s")
        results.append(result)

# Klasteryzacja na danych po redukcj
print("\n--- Klasteryzacja po redukcji wymiarów ---\n")
for method_name in method_names:
    if method_name == "LDA":
        n_components_method = 9
    else:
        n_components_method = n_components

    print(f"Rozpoczynam obliczenia dla: {method_name} (redukcja do {n_components_method} wymiarów)")
    X_transformed, reduction_time = reduce_dimensionality_to_k(X_train, y_train, method_name, n_components_method)
    print(f"Zakończono redukcję wymiarów. Czas: {reduction_time:.2f}s, Wymiary po redukcji: {X_transformed.shape[1]}")

    # Wizualizacja dla t-SNE i UMAP
    if method_name in ["t-SNE", "UMAP"] and X_transformed.shape[1] <= 3:
        if X_transformed.shape[1] == 3:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            scatter = ax.scatter(X_transformed[:, 0], X_transformed[:, 1], X_transformed[:, 2], c=y_train, cmap='viridis')
            legend1 = ax.legend(*scatter.legend_elements(), title="Classes")
            ax.add_artist(legend1)
            plt.title(f"{method_name} - Redukcja do 3 wymiarów")
            plt.show()

    for clustering_method in clustering_methods:
        print(f"Rozpoczynam klasteryzację: {clustering_method}")
        result = cluster_and_evaluate(
            X_transformed, y_train, clustering_method, f"{method_name} ({n_components_method} wym.)", skip_silhouette_on_raw
        )
        print(f"Zakończono klasteryzację. Czas: {result['Clustering Time (s)']:.2f}s")
        result["Reduction Method"] = f"{method_name} ({n_components_method} wym.)"
        result["Reduction Time (s)"] = reduction_time
        results.append(result)

# Wyniki
results_df = pd.DataFrame(results).drop("y_pred", axis=1)
display(Markdown("### Zbiorcze wyniki klasteryzacji"))
display(results_df)

#### Opis

Przejdzmy do kolejnego zagadnienia związanego z redukcją wymiarów dla danych w zbiorze treningowym. . Celem eksperymentu było sprawdzenie, jak dobrze te metody radzą sobie z separacją klastrów odpowiadających poszczególnym cyfrom. 
* PCA (Principal Component Analysis): Podstawowaa lliniowa metoda redukcji wymiarowości, która identyfikuje kierunki największej wariancji w danych.
* KernelPCA (Kernel Principal Component Analysis): Wersja PCA  w któej zasotosowano jądro kosinusowego, umożliwiająca nieliniową redukcję wymiarowości.
* LLE (Locally Linear Embedding): Nieliniowa metoda, stara się zachować lokalne relacje sąsiedztwa między punktami danych.
* LDA (Linear Discriminant Analysis): Metoda nadzorowana, która szuka liniowej kombinacji cech najlepiej separujących klasy (w tym przypadku cyfry).
* t-SNE (t-distributed Stochastic Neighbor Embedding): Nieliniowa metoda, która skupia się na zachowaniu lokalnych struktur i tworzeniu wyraźnie oddzielonych klastrów.
* UMAP (Uniform Manifold Approximation and Projection): Nieliniowa metoda, która stara się zachować zarówno lokalną, jak i globalną strukturę danych.

Do oceny jakości redukcji wymiarowości użyto następujących metryk:

* Silhouette Score: Miara jakości separacji klastrów (dla metod nienadzorowanych).
* Homogeneity, Completeness, V-measure: Miary zgodności wyników klasteryzacji K-means z prawdziwymi etykietami.

Na podstawie uzyskanych wyników oraz wizualizacji UMAP okazał się najskuteczniejszą metodą. Osiągnął on najwyższy silhouette score (0.462), co wskazuje na najlepszą separację klastrów w przestrzeni dwuwymiarowej. Ponadto, UMAP uzyskał najwyższe wartości homogeneity (0.855), completeness (0.865) i V-measure (0.860), co świadczy o najlepszej zgodności wyników klasteryzacji z prawdziwymi etykietami. Czas obliczeniowy UMAP (34.11 s) był stosunkowo krótki. UMAP zapewanił dobrą separację klastrów dla większości cyfr. Widoczne są punkty/etykiety, które znajdują się w skupiskach innych cyfr. Może to wskazywać na outliery, błędy w etykietach nietypowy sposób pisania danej cyfry, lub ograniczenia wynikające z redukcji do dwóch wymiarów.



### Testy statystyczne

In [ ]:
from scipy.stats import (
    bartlett,
    levene,
    fligner,
    f_oneway,
    kruskal,
    shapiro,  # Dodany test Shapiro-Wilka
    anderson  # Dodany test Andersona-Darlinga
)
from IPython.display import Markdown, display
import time
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from statsmodels.graphics.gofplots import qqplot


def reduce_dimensionality(X_train, y_train, method_name, n_components=2):
    """
    Redukuje wymiarowość danych za pomocą określonej metody (PCA lub UMAP).

    Args:
        X_train: Dane treningowe.
        y_train: Etykiety danych treningowych.
        method_name: Nazwa metody redukcji wymiarów ('PCA' lub 'UMAP').
        n_components: Liczba komponentów do zachowania.

    Returns:
        X_transformed: Dane po redukcji wymiarów.
        time_taken: Czas obliczeń.
    """

    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    start_time = time.time()

    if method_name == "PCA":
        method = PCA(n_components=n_components)
        X_transformed = method.fit_transform(X_train_flat)

        # Informacje o PCA są dostępne tylko po dopasowaniu
        if n_components == 'mle':
            n_components_actual = method.n_components_
            print(f"  Liczba komponentów wybrana przez MLE: {n_components_actual}")
        else:
            n_components_actual = n_components

        explained_variance_ratio = method.explained_variance_ratio_
        print(f"  Wyjaśniona wariancja przez {n_components_actual} składowe: {explained_variance_ratio.sum() * 100:.2f}%")

    elif method_name == "UMAP":
        method = umap.UMAP(n_components=n_components, random_state=42)
        X_transformed = method.fit_transform(X_train_flat)

    else:
        raise ValueError(f"Nieznana metoda: {method_name}")

    end_time = time.time()
    time_taken = end_time - start_time

    return X_transformed, time_taken

def analyze_covariance(X, method_name):
    """
    Oblicza i zwraca miary związane z macierzą kowariancji danych X.

    Args:
        X: Dane (po redukcji wymiarów).
        method_name: Nazwa metody redukcji wymiarów.

    Returns:
        Ramkę danych (DataFrame) z wynikami.
    """
    total_variance = np.sum(np.var(X, axis=0))

    results = {
        "Method": method_name,
        "Total Variance": total_variance,
    }

    return pd.DataFrame([results])

def perform_statistical_tests(X_reduced, y, method_name, n_components):
    """
    Przeprowadza testy statystyczne na danych X_reduced pogrupowanych według etykiet y.

    Args:
        X_reduced: Dane po redukcji wymiarów.
        y: Etykiety.
        method_name: Nazwa metody redukcji wymiarów.
        n_components: Liczba komponentów użytych do redukcji

    Returns:
        Ramki danych z wynikami testów.
    """
    groups = [X_reduced[y == label] for label in np.unique(y)]
    groups = [group for group in groups if group.shape[0] > 1]

    if len(groups) < 2:
        print(f"Za mało grup do przeprowadzenia testów statystycznych dla metody {method_name}. Pozostało {len(groups)} grup.")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Test normalności - Anderson-Darling
    normality_test_name = "Anderson-Darling"
    normality_results = []
    for i, group in enumerate(groups):
        result = anderson(group.flatten())  # Spłaszczamy grupę do 1D
        normality_results.append({"statistic": result.statistic, "critical_values": result.critical_values, "significance_levels": result.significance_level})

    # Testy homogeniczności wariancji
    homogeneity_tests = {"Bartlett": bartlett, "Levene": levene, "Fligner-Killeen": fligner}
    homogeneity_results = {}
    for name, test in homogeneity_tests.items():
        try:
            stat, p = test(*groups)
            homogeneity_results[name] = {"statistic": stat, "p-value": p}
        except ValueError as e:
            print(f"Błąd w teście {name}: {e}")
            homogeneity_results[name] = {"statistic": np.nan, "p-value": np.nan}

    # Testy różnic między grupami
    # Kruskal-Wallis - nieparametryczny, nie zakłada normalności
    difference_tests = {"Kruskal-Wallis": kruskal}
    difference_results = {}
    for name, test in difference_tests.items():
        try:
            stat, p = test(*groups)
            difference_results[name] = {"statistic": stat, "p-value": p}
        except ValueError as e:
            print(f"Błąd w teście {name}: {e}")
            difference_results[name] = {"statistic": np.nan, "p-value": np.nan}

    # Tworzenie ramek danych z wynikami
    normality_df = pd.DataFrame(
        [
            f"Stat: {res['statistic']:.3f}, Crit. values: {res['critical_values']}, Sig. levels: {res['significance_levels']}"
            for res in normality_results
        ],
        index=[f"Group {i}" for i in range(len(groups))],
        columns=[normality_test_name]
    )

    homogeneity_df = pd.DataFrame(homogeneity_results).T
    homogeneity_df.index.name = "Test"
    homogeneity_df.reset_index(inplace=True)

    difference_df = pd.DataFrame(difference_results).T
    difference_df.index.name = "Test"
    difference_df.reset_index(inplace=True)

    # Wizualizacja rozkładów dla pierwszych dwóch wymiarów
    if n_components >=2:
        for i, group in enumerate(groups):
            plt.figure(figsize=(12, 4))
            # Histogram
            plt.subplot(1, 2, 1)
            plt.hist(group[:, 0], bins=20)
            plt.title(f"Histogram - {method_name} - Group {i} - Dim 1")

            # Q-Q plot
            plt.subplot(1, 2, 2)
            qqplot(group[:, 0], line='s', ax=plt.gca())
            plt.title(f"Q-Q Plot - {method_name} - Group {i} - Dim 1")

            plt.tight_layout()
            plt.show()

    return normality_df, homogeneity_df, difference_df

# Uruchomienie analizy
if __name__ == "__main__":
    method_names = ["PCA", "UMAP"]
    n_components_pca = 500
    n_components_umap = 500

    covariance_results = pd.DataFrame()
    normality_tests_results = {}
    homogeneity_tests_results = {}
    difference_tests_results = {}

    for method_name in method_names:
        if method_name == "PCA":
            n_components = n_components_pca
        elif method_name == "UMAP":
            n_components = n_components_umap
        
        print(f"Metoda: {method_name}, Liczba komponentów: {n_components}")
        X_transformed, reduction_time = reduce_dimensionality(
            X_train, y_train, method_name, n_components=n_components
        )
        print(f"Czas redukcji wymiarowości: {reduction_time:.2f}s")

        # Analiza kowariancji
        cov_analysis_df = analyze_covariance(X_transformed, method_name)
        covariance_results = pd.concat([covariance_results, cov_analysis_df], ignore_index=True)

        # Testy statystyczne
        normality_df, homogeneity_df, difference_df = perform_statistical_tests(X_transformed, y_train, method_name, n_components)
        normality_tests_results[f"{method_name}_{n_components}"] = normality_df
        homogeneity_tests_results[f"{method_name}_{n_components}"] = homogeneity_df
        difference_tests_results[f"{method_name}_{n_components}"] = difference_df

        # Wizualizacja dla 2 wymiarów (tylko UMAP)
        if method_name == "UMAP":
            plt.figure(figsize=(8, 6))
            for label in np.unique(y_train):
                plt.scatter(X_transformed[y_train == label, 0], X_transformed[y_train == label, 1], label=label)
            plt.title(f"Wizualizacja - {method_name} ({n_components} komponenty)")
            plt.xlabel("Wymiar 1")
            plt.ylabel("Wymiar 2")
            plt.legend()
            plt.show()

    # Wyniki
    display(Markdown("### Analiza kowariancji"))
    display(covariance_results)

    display(Markdown("### Testy normalności (Anderson-Darling)"))
    for key, normality_df in normality_tests_results.items():
        display(Markdown(f"#### {key}"))
        display(normality_df)

    display(Markdown("### Testy homogeniczności wariancji"))
    for key, homogeneity_df in homogeneity_tests_results.items():
        display(Markdown(f"#### {key}"))
        display(homogeneity_df)

    display(Markdown("### Testy różnic między grupami (Kruskal-Wallis)"))
    for key, difference_df in difference_tests_results.items():
        display(Markdown(f"#### {key}"))
        display(difference_df)

### Wizaualizacja sąsiadów outlinerów

In [ ]:
from sklearn.neighbors import NearestNeighbors

def calculate_centroids(data, labels):
    """Oblicza centroidy dla każdej klasy."""
    unique_labels, inverse = np.unique(labels, return_inverse=True)
    counts = np.bincount(inverse)
    sums = np.zeros((len(unique_labels), data.shape[1]))
    np.add.at(sums, inverse, data)
    return sums / counts[:, None]

def find_outlier_indices(X_transformed, y_train, threshold_factor=2.0):
    """
    Automatycznie znajduje punkty, które są daleko od centroidu swojej klasy
    i blisko centroidu innej klasy w przestrzeni zredukowanej.
    Zwraca listę krotek (indeks, odległość do centroidu swojej klasy, odległość do najbliższego centroidu innej klasy).
    """
    centroids_reduced = calculate_centroids(X_transformed, y_train)
    distances = np.linalg.norm(X_transformed[:, None, :] - centroids_reduced[None, :, :], axis=2)

    outlier_data = []
    for label in np.unique(y_train):
        indices_this_label = np.where(y_train == label)[0]
        distances_to_own_centroid = distances[indices_this_label, label]

        threshold = np.mean(distances_to_own_centroid) + threshold_factor * np.std(distances_to_own_centroid)

        distances_copy = distances[indices_this_label, :].copy()
        distances_copy[:, label] = np.inf
        min_distances_to_other_centroid = np.min(distances_copy, axis=1)

        outlier_indices_this_label = indices_this_label[
            (distances_to_own_centroid > threshold)
        ]

        outlier_data.extend(
            list(
                zip(
                    outlier_indices_this_label,
                    distances_to_own_centroid[distances_to_own_centroid > threshold],
                    min_distances_to_other_centroid[distances_to_own_centroid > threshold],
                )
            )
        )

    return outlier_data

def analyze_outlier(X_train, y_train, X_transformed, outlier_data, knn, method_name="UMAP", max_outliers=10):
    """
    Analizuje pojedynczy outlier: wyświetla obraz, odległości do centroidów
    i etykiety najbliższych sąsiadów.
    Ogranicza liczbę analizowanych outlierów do max_outliers.
    """
    centroids_original = calculate_centroids(X_train, y_train)

    print(f"Analiza outlierów dla metody {method_name}:")

    for i, (index, dist_own, dist_other) in enumerate(outlier_data):
        if i >= max_outliers:
            break

        label = y_train[index]
        dist_original_own = np.linalg.norm(X_train[index] - centroids_original[label])

        print(f"\nIndeks: {index}, Etykieta: {label}")
        print(f"Odległość do centroidu swojej klasy (przestrzeń oryginalna): {dist_original_own:.2f}")
        print(f"Odległość do centroidu swojej klasy (przestrzeń zredukowana): {dist_own:.2f}")
        print(f"Minimalna odległość do centroidu innej klasy (przestrzeń zredukowana): {dist_other:.2f}")

        distances, indices = knn.kneighbors(X_train[index][np.newaxis, :])
        print("\nIndeksy najbliższych sąsiadów:", indices[0])
        print("Etykiety najbliższych sąsiadów:", y_train[indices[0]])

        plt.figure(figsize=(10, 5))
        for j in range(6):
            plt.subplot(1, 6, j + 1)
            if j == 0:
                img = X_train[index].reshape(28, 28)
                title = f"Outlier\nIndeks: {index}\nEtykieta: {label}"
            else:
                neighbor_index = indices[0][j - 1]
                img = X_train[neighbor_index].reshape(28, 28)
                title = f"Sąsiad: {y_train[neighbor_index]}"
            plt.imshow(img, cmap="gray")
            plt.title(title)
            plt.axis("off")

        plt.suptitle(f"Analiza Outliera dla metody {method_name}")
        plt.tight_layout()
        plt.show()

# Spłaszczenie danych
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Redukcja wymiarów UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
X_transformed = reducer.fit_transform(X_train_flat)

#Wizualizacja
df = pd.DataFrame(X_transformed, columns=["UMAP wymiar 1", "UMAP wymiar 2"])
df["Cyfra"] = y_train

plt.figure(figsize=(12, 10))
sns.scatterplot(
    x="UMAP wymiar 1",
    y="UMAP wymiar 2",
    hue="Cyfra",
    data=df,
    palette="viridis",
    alpha=0.5,
    legend="full",
)
plt.title("UMAP MNIST")
plt.show()


knn = NearestNeighbors(n_neighbors=6, algorithm="auto")
knn.fit(X_train_flat)

# Znajdowanie outlierów na podstawie odległości do centroidów
outlier_data = find_outlier_indices(X_transformed, y_train, threshold_factor=2.5)

# Analiza outlierów
analyze_outlier(X_train_flat, y_train, X_transformed, outlier_data, knn, method_name="UMAP", max_outliers=10)

### One hot encoding

In [ ]:
# Reshape 
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

# One-hot encoding
encoder = OneHotEncoder()
y_train_ENC = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_ENC = encoder.transform(y_test.reshape(-1, 1)).toarray()

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_train_ENC:", y_train_ENC.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of y_test_ENC:", y_test_ENC.shape)

### Normalizacja histogramem

In [ ]:
X_train = cv2.normalize(X_train, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_64F)
X_train = X_train.astype(np.uint8)
# Wyrównywanie histogramu
for i in range(X_train.shape[0]):
    X_train[i] = cv2.equalizeHist(X_train[i])

X_test = cv2.normalize(X_test, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_64F)
X_test = X_test.astype(np.uint8)
for i in range(X_test.shape[0]):
    X_test[i] = cv2.equalizeHist(X_test[i])

### Powiększanie i wygładzenie obrazu

In [ ]:
def enlarge_and_smooth(image, new_size):
    """Powiększa obraz za pomocą interpolacji bicubic."""
    enlarged_image = cv2.resize(image, new_size, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

# # Sztuczne zwiększenie rozmiaru obrazu 
# sample_indices = np.random.choice(X_train.shape[0], 5, replace=False)

# # Dla każdego wybranego obrazu
# for idx in sample_indices:
#     # Oryginalny obraz
#     original_image = X_train[idx]

#     # Powiększony i wygładzony obraz
#     enlarged_smoothed_image = enlarge_and_smooth(original_image, new_size=(42, 42))

#     # Wyświetl oryginalny i przekształcony obraz obok siebie
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
#     print("Shape of original_image:", original_image.shape)
#     print("Shape of enlarged_smoothed_image:", enlarged_smoothed_image.shape)

#     sns.heatmap(original_image, cmap="viridis", ax=axes[0]) 
#     axes[0].set_title(f'Oryginalny obraz ({idx})')

#     sns.heatmap(enlarged_smoothed_image, cmap="viridis", ax=axes[1]) 
#     axes[1].set_title(f'Powiększony i wygładzony obraz (Lanczos4) ({idx})')
#     plt.show()

### Split na danych treningowych

In [ ]:
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(
    X_train, y_train, test_size=0.2, random_state=69)
print("Rozmiar X_train_base:", X_train_base.shape)
print("Rozmiar X_val_base:", X_val_base.shape)
print("Rozmiar y_train_base:", y_train_base.shape)
print("Rozmiar y_val_base:", y_val_base.shape)

### Upscalowanie danych
smoothed_X_train = np.array([enlarge_and_smooth(image, new_size=(42, 42)) for image in X_train])

X_train_Upsc, X_val_Upsc, y_train_Upsc, y_val_Upsc = train_test_split(
    smoothed_X_train, y_train, test_size=0.2, random_state=69)
print("Rozmiar X_train_Upsc:", X_train_Upsc.shape)
print("Rozmiar X_val_Upsc:", X_val_Upsc.shape)
print("Rozmiar y_train_Upsc:", y_train_Upsc.shape)
print("Rozmiar y_val_Upsc:", y_val_Upsc.shape)

### Upscalowanie danych testowych
X_test = np.array([enlarge_and_smooth(image, new_size=(42, 42)) for image in X_test])
print("Rozmiar X_test:", X_test.shape)


## Klasa Ewaulacji modelu

In [ ]:
def flatten(X: np.ndarray) -> np.ndarray:
    return X.reshape(X.shape[0], -1)

# Transformer do użycia w pipeline
flatten_transformer = FunctionTransformer(flatten)

class ModelEvaluator:
    """
    Klasa do ewaluacji modeli klasyfikacji.
    Umożliwia przekazanie pipeline'u, opcjonalne dodanie kroku spłaszczania,
    oblicza wybrane metryki (logika sterowana danymi) z konfigurowalnym trybem uśredniania,
    i (jeśli verbose=True) wyświetla wyniki oraz generuje obiekty wykresów.
    """

    def __init__(
        self,
        metrics: Optional[List[str]] = None,
        average_mode: Literal["macro", "micro", "weighted", "binary"] = "macro",
        display_labels: Optional[List[str]] = None,
    ):
        """
        Inicjalizuje ModelEvaluator.
        (Opis Args bez zmian)
        """
        self.metrics = metrics if metrics is not None else [
            "accuracy", "precision", "recall", "f1"
        ]
        # Walidacja average_mode
        valid_average_modes = ["macro", "micro", "weighted", "binary", None] # Dodano None jako możliwy (choć nie domyślny)
        if average_mode not in valid_average_modes:
            raise ValueError(f"Nieprawidłowy average_mode. Wybierz jeden z: {valid_average_modes}")
        self.average_mode = average_mode
        self.display_labels = display_labels

        self._metric_calculators: Dict[str, Tuple[Callable, Dict[str, Any]]] = {
            "accuracy": (accuracy_score, {}), # Accuracy nie potrzebuje dodatkowych arg.
            "precision": (precision_score, {"average": self.average_mode, "zero_division": 0}),
            "recall": (recall_score, {"average": self.average_mode, "zero_division": 0}),
            "f1": (f1_score, {"average": self.average_mode, "zero_division": 0}),
        }

        # Sprawdź, czy wymagane metryki są obsługiwane (oparte na kluczach mapowania + roc_auc)
        supported_metrics = set(self._metric_calculators.keys()) | {"roc_auc"}
        for metric in self.metrics:
            if metric not in supported_metrics:
                warnings.warn(f"Metryka '{metric}' nie jest bezpośrednio obsługiwana przez tę klasę.")

    def evaluate(
        self,
        pipeline: Pipeline,
        X_train: np.ndarray,
        y_train: np.ndarray,
        X_val: np.ndarray,
        y_val: np.ndarray,
        add_flatten_step: bool = False,
        calculate_proba: bool = False,
        roc_pos_label: Union[int, str] = 1,
        verbose: bool = True,
    ) -> Dict[str, Any]:
        """
        Trenuje i ewaluuje podany pipeline na danych walidacyjnych.
        """
        eval_pipeline = pipeline

        if add_flatten_step:
            if verbose:
                print("INFO: Dodaję krok 'flatten' na początku pipeline'u.")
            eval_pipeline = Pipeline([
                ("flatten", flatten_transformer),
                *pipeline.steps
            ])

        # Trenowanie pipeline'u
        eval_pipeline.fit(X_train, y_train)

        # Predykcje
        y_pred = eval_pipeline.predict(X_val)

        # Prawdopodobieństwa
        y_prob = None
        if calculate_proba and "roc_auc" in self.metrics:
            y_prob = self._get_probabilities(eval_pipeline, X_val, verbose)

        # Obliczanie metryk (używa zrefaktoryzowanej metody)
        metric_results = self._calculate_metrics(y_val, y_pred, y_prob, roc_pos_label, calculate_proba)

        # Przygotowanie wyników do zwrócenia
        output: Dict[str, Any] = {}
        metrics_df = pd.DataFrame([metric_results]) # Utwórz DF z wynikami metryk
        output['metrics'] = metrics_df

        if verbose:
            self._display_verbose_output(
                y_val, y_pred, y_prob, roc_pos_label, eval_pipeline, metrics_df, metric_results, output
            )
        return output

    # --- Metody pomocnicze dla Refaktoryzacji 2 ---
    def _get_model_name(self, eval_pipeline: Union[Pipeline, BaseEstimator]) -> str:
        """Pobiera nazwę ostatniego kroku pipeline'u lub nazwę modelu."""
        if isinstance(eval_pipeline, Pipeline):
             if not eval_pipeline.steps: return "Pusty Pipeline"
             final_estimator = eval_pipeline.steps[-1][1]
             if isinstance(final_estimator, str) and final_estimator == "passthrough":
                 if len(eval_pipeline.steps) > 1:
                      final_estimator = eval_pipeline.steps[-2][1]
                 else:
                      return "Pipeline (tylko passthrough?)"
             return final_estimator.__class__.__name__
        else:
             return eval_pipeline.__class__.__name__

    def _display_metrics_table(self, metrics_df: pd.DataFrame):
        """Wyświetla tabelę z metrykami."""
        print("\nMetryki:")
        try:
            display(metrics_df)
        except ImportError:
            print(metrics_df.to_string())

    def _display_classification_report(self, y_true: np.ndarray, y_pred: np.ndarray) -> pd.DataFrame:
        """Formatuje i wyświetla raport klasyfikacji, zwraca DataFrame raportu."""
        print("\nRaport Klasyfikacji:")
        report_df = self._format_classification_report(y_true, y_pred)
        try:
            display(report_df)
        except ImportError:
            print(report_df.to_string())
        return report_df

    def _generate_and_collect_plots(
        self,
        y_true: np.ndarray,
        y_pred: np.ndarray,
        y_prob: Optional[np.ndarray],
        roc_pos_label: Union[int, str],
        metric_results: Dict[str, Optional[float]],
        output_dict: Dict[str, Any] 
    ):
        """Generuje obiekty wykresów i zapisuje je w słowniku wyjściowym."""
        output_dict['figures'] = {}
        print("\nGenerowanie wykresów (obiekty zwrócone w słowniku 'figures'):")

        # Macierz pomyłek
        try:
            fig_cm, ax_cm = self._plot_confusion_matrix(y_true, y_pred)
            output_dict['figures']['confusion_matrix'] = (fig_cm, ax_cm)
            print("- Wygenerowano macierz pomyłek.")
        except Exception as e:
            warnings.warn(f"Nie udało się wygenerować macierzy pomyłek: {e}")

        # Krzywa ROC
        if "roc_auc" in metric_results and metric_results["roc_auc"] is not None and y_prob is not None:
            try:
                roc_data = self._calculate_roc_data(y_true, y_prob, roc_pos_label)
                if roc_data:
                    auc_score, fpr, tpr = roc_data
                    fig_roc, ax_roc = self._plot_roc_curve(fpr, tpr, auc_score)
                    output_dict['figures']['roc_curve'] = (fig_roc, ax_roc)
                    print("- Wygenerowano krzywą ROC.")
            except Exception as e:
                 warnings.warn(f"Nie udało się wygenerować krzywej ROC: {e}")
        elif "roc_auc" in self.metrics: 
             print("- Nie można wygenerować krzywej ROC (brak danych ROC lub prawdopodobieństw).")


    def _display_verbose_output(
        self,
        y_true: np.ndarray,
        y_pred: np.ndarray,
        y_prob: Optional[np.ndarray],
        roc_pos_label: Union[int, str],
        eval_pipeline: Union[Pipeline, BaseEstimator],
        metrics_df: pd.DataFrame,
        metric_results: Dict[str, Optional[float]],
        output_dict: Dict[str, Any] # Słownik do zapisu raportu i figur
    ):
        """Główna metoda pomocnicza do obsługi wyjścia w trybie verbose."""
        print("\n--- Wyniki Ewaluacji ---")
        model_name = self._get_model_name(eval_pipeline)
        print(f"Model (ostatni krok): {model_name}")

        self._display_metrics_table(metrics_df)
        report_df = self._display_classification_report(y_true, y_pred)
        output_dict['classification_report'] = report_df 
        self._generate_and_collect_plots(y_true, y_pred, y_prob, roc_pos_label, metric_results, output_dict)

    def _get_probabilities(
        self,
        model: BaseEstimator,
        X: np.ndarray,
        verbose: bool = True
    ) -> Optional[np.ndarray]:
        """Prywatna metoda do bezpiecznego pobierania prawdopodobieństw."""
        try:
            y_prob_all = model.predict_proba(X)
            if y_prob_all.shape[1] >= 2:
                return y_prob_all[:, 1] 
            elif y_prob_all.shape[1] == 1:
                warnings.warn("Metoda predict_proba zwróciła tylko jedną kolumnę.")
                return y_prob_all[:, 0]
            else:
                warnings.warn("Metoda predict_proba zwróciła pusty wynik.")
                return None
        except AttributeError:
            if verbose:
                warnings.warn("Model nie posiada metody `predict_proba`. Metryka ROC AUC nie zostanie obliczona.")
            return None
        except Exception as e:
            if verbose:
                warnings.warn(f"Wystąpił błąd podczas wywołania `predict_proba`: {e}")
            return None

    def _calculate_metrics(
        self,
        y_true: np.ndarray,
        y_pred: np.ndarray,
        y_prob: Optional[np.ndarray],
        roc_pos_label: Union[int, str],
        calculate_proba: bool
    ) -> Dict[str, Optional[float]]:
        """Prywatna metoda do obliczania wybranych metryk (sterowana danymi)."""
        results: Dict[str, Optional[float]] = {}
        base_args = {"y_true": y_true, "y_pred": y_pred}

        for metric_name in self.metrics:
            if metric_name == "roc_auc":
                continue # Obsługa ROC AUC poniżej

            if metric_name in self._metric_calculators:
                func, specific_args = self._metric_calculators[metric_name]
                try:
                    # Połącz argumenty bazowe i specyficzne dla danej metryki
                    all_args = {**base_args, **specific_args}
                    results[metric_name] = func(**all_args)
                except Exception as e:
                    warnings.warn(f"Nie udało się obliczyć metryki '{metric_name}': {e}")
                    results[metric_name] = None

        # Obsługa ROC AUC
        results["roc_auc"] = None
        if "roc_auc" in self.metrics:
            if calculate_proba:
                if y_prob is not None:
                    roc_data = self._calculate_roc_data(y_true, y_prob, roc_pos_label)
                    if roc_data:
                        results["roc_auc"] = roc_data[0] # Zapisz tylko wartość AUC
                    else:
                        pass
                else:
                     pass 

        return results

    def _calculate_roc_data(
        self,
        y_true: np.ndarray,
        y_prob: np.ndarray,
        pos_label: Union[int, str] = 1
    ) -> Optional[Tuple[float, np.ndarray, np.ndarray]]:
        """Oblicza AUC oraz FPR, TPR dla krzywej ROC."""
        # (Implementacja bez zmian)
        try:
            if len(y_true) != len(y_prob): raise ValueError("Długości y_true i y_prob nie są zgodne.")
            unique_labels = np.unique(y_true)
            if pos_label not in unique_labels:
                warnings.warn(f"Etykieta pozytywna '{pos_label}' nie znaleziona w y_true. Unikalne etykiety: {unique_labels}. Nie można obliczyć ROC AUC.")
                return None
            if len(unique_labels) < 2:
                warnings.warn(f"W y_true znaleziono tylko jedną klasę: {unique_labels}. Nie można obliczyć ROC AUC.")
                return None
            fpr, tpr, _ = roc_curve(y_true, y_prob, pos_label=pos_label)
            roc_auc_score = auc(fpr, tpr)
            return roc_auc_score, fpr, tpr
        except ValueError as e:
            warnings.warn(f"Błąd podczas obliczania ROC: {e}. Sprawdź format danych i `pos_label`.")
            return None
        except Exception as e:
            warnings.warn(f"Nieoczekiwany błąd podczas obliczania ROC: {e}")
            return None

    def _plot_confusion_matrix(
        self, y_true: np.ndarray, y_pred: np.ndarray
    ) -> Tuple[plt.Figure, plt.Axes]:
        """Generuje obiekt wykresu macierzy pomyłek."""
        # (Implementacja bez zmian)
        cm = confusion_matrix(y_true, y_pred)
        labels = self.display_labels if self.display_labels else np.unique(y_true)
        fig, ax = plt.subplots(figsize=(8, 6))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(ax=ax, cmap=plt.cm.Blues, values_format='d')
        ax.set_title("Macierz pomyłek")
        return fig, ax

    def _format_classification_report(
        self, y_true: np.ndarray, y_pred: np.ndarray
    ) -> pd.DataFrame:
        """Generuje raport klasyfikacji jako Pandas DataFrame."""
        # (Implementacja bez zmian)
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
        report_df = pd.DataFrame(report).transpose()
        report_df['support'] = report_df['support'].astype(int)
        report_df[['precision', 'recall', 'f1-score']] = report_df[['precision', 'recall', 'f1-score']].round(3)
        return report_df

    def _plot_roc_curve(
        self, fpr: np.ndarray, tpr: np.ndarray, roc_auc: float
    ) -> Tuple[plt.Figure, plt.Axes]:
        """Generuje obiekt wykresu krzywej ROC."""
        # (Implementacja bez zmian)
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.plot(fpr, tpr, color="darkorange", lw=2, label=f"Krzywa ROC (pole = {roc_auc:.3f})")
        ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--", label="Losowy klasyfikator")
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel("False Positive Rate (FPR)")
        ax.set_ylabel("True Positive Rate (TPR)")
        ax.set_title("Krzywa ROC")
        ax.legend(loc="lower right")
        ax.grid(alpha=0.3)
        return fig, ax

### Sprawdzenie modelu wykorzystując VotingClassifier dla danych orginalnaych

In [ ]:
# --- Definicja modeli  ---
qda = QDA(reg_param=0.1)
hgb = HistGradientBoostingClassifier(random_state=42) 
et = ExtraTreesClassifier(random_state=42) 

# Utworzenie VotingClassifier
VotClass = VotingClassifier(
    estimators=[('qda', qda), ('hgb', hgb), ('et', et)],
    voting='soft'
)

models = [VotClass]

# Tworzenie instancji klasy ModelEvaluator 
evaluator = ModelEvaluator(
    metrics=["accuracy", "precision", "recall", "f1", "roc_auc"], 
    average_mode="macro"
)

all_metrics_data = {}

# Trenowanie i ewaluacja modeli
for model in models:
    if hasattr(model, '__class__') and hasattr(model.__class__, '__name__'):
         model_name = model.__class__.__name__
    else:
         model_name = str(model)[:30]

    print(f"\n--- Ewaluacja modelu: {model_name} ---")

    # Utworzenie pipeline'u
    pipeline = Pipeline([
        ("classifier", model) 
    ])

    # Trenowanie i ewaluacja pipeline'u
    eval_results = evaluator.evaluate(
        pipeline=pipeline,
        X_train=X_train_base,
        y_train=y_train_base,
        X_val=X_val_base,
        y_val=y_val_base,
        add_flatten_step=True,  # Ustaw wg potrzeb danych
        calculate_proba=True,   # Włączone dla roc_auc i voting='soft'
        roc_pos_label=1,        # Dostosuj etykietę klasy pozytywnej
        verbose=True            # Czy drukować wyniki na bieżąco?
    )

    if 'metrics' in eval_results and isinstance(eval_results['metrics'], pd.DataFrame):
        all_metrics_data[model_name] = eval_results['metrics']
    else:
        print(f"OSTRZEŻENIE: Brak DataFrame 'metrics' w wynikach dla modelu {model_name}")

    # Opcjonalnie: figury i raporty
    # if 'figures' in eval_results:
    #     all_figures[model_name] = eval_results['figures']
    # if 'classification_report' in eval_results:
    #     all_reports[model_name] = eval_results['classification_report']

if all_metrics_data:
    summary_metrics_df = pd.concat(all_metrics_data)
    if isinstance(summary_metrics_df.index, pd.MultiIndex):
        summary_metrics_df.index = summary_metrics_df.index.get_level_values(0)

    print("\n--- Zbiorcze Metryki dla Wszystkich Modeli ---")
    try:
        display(summary_metrics_df)
    except ImportError:
        print(summary_metrics_df.to_string())
else:
    print("\nNie zebrano żadnych danych metryk do wyświetlenia.")

### Sprawdzenie modelu wykorzystując VotingClassifier dla danych upscalowanych

In [ ]:
# --- Definicja modeli  ---
qda = QDA(reg_param=0.1)
hgb = HistGradientBoostingClassifier(random_state=42) 
et = ExtraTreesClassifier(random_state=42) 

# Utworzenie VotingClassifier
VotClass = VotingClassifier(
    estimators=[('qda', qda), ('hgb', hgb), ('et', et)],
    voting='soft'
)

models = [VotClass]

# Tworzenie instancji klasy ModelEvaluator 
evaluator = ModelEvaluator(
    metrics=["accuracy", "precision", "recall", "f1", "roc_auc"], 
    average_mode="macro"
)

all_metrics_data = {}

# Trenowanie i ewaluacja modeli
for model in models:
    if hasattr(model, '__class__') and hasattr(model.__class__, '__name__'):
         model_name = model.__class__.__name__
    else:
         model_name = str(model)[:30]

    print(f"\n--- Ewaluacja modelu: {model_name} ---")

    # Utworzenie pipeline'u
    pipeline = Pipeline([
        ("classifier", model) 
    ])

    # Trenowanie i ewaluacja pipeline'u
    eval_results = evaluator.evaluate(
        pipeline=pipeline,
        X_train=X_train_Upsc,
        y_train=y_train_Upsc,
        X_val=X_val_Upsc,
        y_val=y_val_Upsc,
        add_flatten_step=True,  # Ustaw wg potrzeb danych
        calculate_proba=True,   # Włączone dla roc_auc i voting='soft'
        roc_pos_label=1,        # Dostosuj etykietę klasy pozytywnej
        verbose=True            # Czy drukować wyniki na bieżąco?
    )

    if 'metrics' in eval_results and isinstance(eval_results['metrics'], pd.DataFrame):
        all_metrics_data[model_name] = eval_results['metrics']
    else:
        print(f"OSTRZEŻENIE: Brak DataFrame 'metrics' w wynikach dla modelu {model_name}")

    # Opcjonalnie: figury i raporty
    # if 'figures' in eval_results:
    #     all_figures[model_name] = eval_results['figures']
    # if 'classification_report' in eval_results:
    #     all_reports[model_name] = eval_results['classification_report']

if all_metrics_data:
    summary_metrics_df = pd.concat(all_metrics_data)
    if isinstance(summary_metrics_df.index, pd.MultiIndex):
        summary_metrics_df.index = summary_metrics_df.index.get_level_values(0)

    print("\n--- Zbiorcze Metryki dla Wszystkich Modeli ---")
    try:
        display(summary_metrics_df)
    except ImportError:
        print(summary_metrics_df.to_string())
else:
    print("\nNie zebrano żadnych danych metryk do wyświetlenia.")

### Detekcja outlinerów oraz agumentacja danych - Testy Optuna

In [ ]:
try:
    import optunahub
    OPTUNAHUB_AVAILABLE = True
except ImportError:
    OPTUNAHUB_AVAILABLE = False
    print("Ostrzeżenie: Moduł optunahub nie jest zainstalowany. Używam domyślnego TPESampler.")
import warnings
import traceback
import gc
from typing import List, Dict, Optional, Any, Tuple, Union, Literal, Callable
from tqdm.auto import tqdm
import logging

# --- Konfiguracja Logowania ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Konfiguracja Globalna ---
RANDOM_STATE = 42
TARGET_IMG_SIZE = 42

# Konfiguracja Pre-obliczenia UMAP dla LOF/Weak
PRE_UMAP_CONFIG = {
    'n_components': 2,
    'n_neighbors': 30,
    'min_dist': 0.1,
    'metric': 'euclidean',
    'random_state': RANDOM_STATE,
    'n_jobs': 1
}

# Konfiguracja Augmentacji
AUGMENTATION_CONFIG = {
    'padding_val': 35,
    'sobel_thresh': 50,
    'morph_disk_size': 1,
    'affine_scale_range': (0.97, 1.04),
    'affine_rot_range_deg': (-18, 19),
    'affine_shear_range_deg': (-5, 6),
    'affine_trans_range': (-5, 6)
    # n_augmentations_per_sample będzie optymalizowane
}

# Konfiguracja Optuny
OPTUNA_CONFIG = {
    'n_trials_umap': 50, # Dostosuj
    'n_trials_no_umap': 500,# Dostosuj
    'n_jobs': 10,# WAŻNE: Zacznij od 1 dla debugowania!
    'db_path': f"sqlite:///study_Refactored_v2.db", # Dynamiczna nazwa bazy
    'study_name_prefix_umap': "Aug_w_UMAP_Refactored",
    'study_name_prefix_no_umap': "Aug_no_UMAP_Refactored"
}

# Flaga kontrolująca, które study uruchomić
RUN_WITH_UMAP_IN_PIPELINE = False # Ustaw na True lub False

# Ignorowanie ostrzeżeń
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING) # Zmniejsz gadatliwość Optuny

# --- Funkcja Flatten i Transformer ---
def flatten(X: np.ndarray) -> np.ndarray:
    if X.ndim <= 2: return X
    return X.reshape(X.shape[0], -1)
flatten_transformer = FunctionTransformer(flatten)

# --- Inicjalizacja Ewaluatora ---
evaluator = ModelEvaluator(
    metrics=["accuracy", "precision", "recall", "f1"],
    average_mode="macro"
)

# --- Callback dla Optuny ---
def callback(study, trial):
    value_str = f"{trial.value:.6f}" if trial.value is not None else "None (błąd?)"
    logging.info(f"Trial {trial.number} zakończony. Wartość: {value_str}.")

# --- Funkcje Pomocnicze Identyfikacji Indeksów ---
def calculate_centroids(X, y):
    classes = np.unique(y)
    centroids, valid_classes = [], []
    for k in classes:
        Xk = X[y == k]
        if Xk.size > 0 and np.issubdtype(Xk.dtype, np.number):
             if Xk.ndim == 1: Xk = Xk.reshape(-1, 1)
             if Xk.ndim > 1:
                try: centroids.append(Xk.mean(axis=0)); valid_classes.append(k)
                except Exception as e: logging.warning(f"Nie obliczono centroidu dla klasy {k}: {e}")
    return np.array(centroids), np.array(valid_classes)

def find_weakly_described_indices(X_transformed: np.ndarray, y_train: np.ndarray, threshold_factor: float) -> np.ndarray:
    logging.info("      - Uruchamiam find_weakly_described_indices...")
    try:
        weakly_described_indices = []
        unique_labels_train = np.unique(y_train)
        if not np.issubdtype(X_transformed.dtype, np.number): logging.warning("X_transformed nie numeryczne."); return np.array([], dtype=int)
        if not np.issubdtype(y_train.dtype, np.number): logging.warning("y_train nie numeryczne."); return np.array([], dtype=int)
        centroids_reduced, centroid_labels = calculate_centroids(X_transformed, y_train)
        if centroids_reduced.shape[0] < 2: logging.warning(f"Znaleziono {centroids_reduced.shape[0]} centroidów."); return np.array([], dtype=int)
        if X_transformed.shape[1]!=centroids_reduced.shape[1]: logging.warning("Niezgodność wymiarów X i centroidów."); return np.array([],dtype=int)

        label_to_centroid_idx = {label: idx for idx, label in enumerate(centroid_labels)}
        distances = np.linalg.norm(X_transformed.astype(np.float32)[:, None, :] - centroids_reduced.astype(np.float32)[None, :, :], axis=2)

        for label in unique_labels_train:
            if label not in label_to_centroid_idx: continue
            centroid_idx_for_label = label_to_centroid_idx[label]
            indices_this_label = np.where(y_train == label)[0]
            if indices_this_label.size == 0: continue
            distances_subset = distances[indices_this_label, :]
            if distances_subset.shape[1] < 2: continue
            distances_copy = distances_subset.copy()
            distances_copy[:, centroid_idx_for_label] = np.inf
            min_distances_to_other_centroid = np.min(distances_copy, axis=1)
            mean_dist = np.mean(min_distances_to_other_centroid)
            threshold = 1e-9 if mean_dist < 1e-9 else mean_dist / threshold_factor
            weakly_described_indices_this_label = indices_this_label[ (min_distances_to_other_centroid < threshold) ]
            weakly_described_indices.extend(weakly_described_indices_this_label)

        result_indices = np.unique(np.array(weakly_described_indices)).astype(int)
        logging.info(f"      - find_weakly_described_indices zakończono ({len(result_indices)} słabo opisanych).")
        return result_indices
    except Exception as e:
        logging.error(f"BŁĄD w find_weakly_described_indices: {e}", exc_info=True)
        return np.array([], dtype=int)

def _detect_outliers(X_train: np.ndarray, X_train_reduced_for_lof: np.ndarray, n_neighbors_lof: int, contamination_lof: float, contamination_forest: float) -> np.ndarray:
    logging.info("      - Rozpoczynam detekcję outlierów (LOF + IF)...")
    y_pred_lof = np.ones(X_train.shape[0], dtype=int)
    y_pred_iso = np.ones(X_train.shape[0], dtype=int)
    # LOF
    logging.info("        - Uruchamiam LOF...")
    actual_n_neighbors_lof = min(n_neighbors_lof, X_train_reduced_for_lof.shape[0] - 1)
    if actual_n_neighbors_lof <= 0: logging.warning("Za mało próbek dla LOF.")
    else:
        lof = LocalOutlierFactor(n_neighbors=actual_n_neighbors_lof, contamination=contamination_lof, n_jobs=-1)
        try: y_pred_lof = lof.fit_predict(X_train_reduced_for_lof); logging.info(f"LOF OK ({np.sum(y_pred_lof == -1)}).")
        except Exception as e: logging.error(f"BŁĄD LOF: {e}")
    # Isolation Forest
    logging.info("        - Uruchamiam IF...")
    iso_forest = IsolationForest(contamination=contamination_forest, random_state=RANDOM_STATE, n_jobs=-1)
    try:
        X_flat = flatten(X_train); y_pred_iso = iso_forest.fit_predict(X_flat); del X_flat; gc.collect()
        logging.info(f"IF OK ({np.sum(y_pred_iso == -1)}).")
    except Exception as e: logging.error(f"BŁĄD IF: {e}")
    # Kombinacja OR
    outlier_indices = np.where((y_pred_lof == -1) | (y_pred_iso == -1))[0]
    logging.info(f"      - Detekcja outlierów zakończona. Znaleziono: {len(outlier_indices)}.")
    return outlier_indices

# --- Klasa Augmentacji Obrazów ---
class ImageAugmenter:
    def __init__(self, config: Dict = AUGMENTATION_CONFIG, target_size: int = TARGET_IMG_SIZE):
        self.config = config
        self.target_size = target_size
        logging.info("Inicjalizacja ImageAugmenter zakończona.")

    def _apply_affine(self, image: np.ndarray) -> np.ndarray:
        padded = np.pad(image, pad_width=self.config['padding_val'], mode='constant', constant_values=0)
        transform = AffineTransform(
            scale=np.random.uniform(*self.config['affine_scale_range'], size=2),
            rotation=np.deg2rad(np.random.randint(*self.config['affine_rot_range_deg'])),
            shear=np.deg2rad(np.random.randint(*self.config['affine_shear_range_deg'])),
            translation=np.random.randint(*self.config['affine_trans_range'], size=2))
        # Używaj float64 dla warp, potem clip i konwersja
        warped = warp(padded.astype(np.float64)/255.0, transform, mode='constant', cval=0, preserve_range=False, output_shape=padded.shape)
        return np.clip(warped * 255, 0, 255).astype(np.uint8)

    def _extract_digit(self, image: np.ndarray) -> Optional[np.ndarray]:
        try:
            if image.ndim == 3: image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Zwiększony kernel Sobela
            gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5); gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
            mag = cv2.magnitude(gx, gy)
            if np.max(mag) <= 1e-6: return None
            mag_norm = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            _, thresh = cv2.threshold(mag_norm, self.config['sobel_thresh'], 255, cv2.THRESH_BINARY)
            contours = find_contours(thresh, 0.5)
            if not contours: return None
            # Znajdź największy kontur wg pola bounding box
            largest_contour = max(contours, key=lambda c: np.prod(np.max(c, axis=0) - np.min(c, axis=0)))
            minr, minc = np.min(largest_contour, axis=0).astype(int)
            maxr, maxc = np.max(largest_contour, axis=0).astype(int)
            if minr >= maxr or minc >= maxc: return None
            isolated = image[minr:maxr+1, minc:maxc+1]
            return isolated if isolated.size > 0 else None
        except Exception: return None

    def _apply_morph(self, image: Optional[np.ndarray]) -> Optional[np.ndarray]:
        if image is None or image.size == 0: return None
        op_choice = np.random.choice([None, 'erosion', 'opening', 'closing'])
        if op_choice is None: return image
        try:
            kernel = disk(self.config['morph_disk_size'])
            if op_choice == 'erosion': result = erosion(image, kernel)
            elif op_choice == 'opening': result = opening(image, kernel)
            else: result = closing(image, kernel) # closing
            return result if result.size > 0 else None
        except Exception: return None

    def _resize_center(self, image: Optional[np.ndarray]) -> Optional[np.ndarray]:
        if image is None or image.size == 0: return None
        try:
            if image.dtype != np.uint8: image = np.clip(image, 0, 255).astype(np.uint8)
            h, w = image.shape[:2]
            if h <= 0 or w <= 0: return None
            if h > self.target_size or w > self.target_size:
                scale = min(self.target_size / h, self.target_size / w)
                nh, nw = max(1, int(h * scale)), max(1, int(w * scale))
                image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_AREA)
                h, w = image.shape[:2]

            canvas = np.zeros((self.target_size, self.target_size), dtype=np.uint8)
            r_off = max(0, (self.target_size - h) // 2); c_off = max(0, (self.target_size - w) // 2)
            ph = min(h, self.target_size - r_off); pw = min(w, self.target_size - c_off)
            src_frag = image[:ph, :pw]
            target_slice = (slice(r_off, r_off + ph), slice(c_off, c_off + pw))
            if canvas[target_slice].shape == src_frag.shape:
                canvas[target_slice] = src_frag; return canvas
            else: return None
        except Exception: return None

    def augment_batch(self, X_train, y_train, indices_to_augment, n_augmentations_per_sample):
        """ Główna metoda augmentacji dla podanych indeksów. """
        if not isinstance(indices_to_augment, np.ndarray) or indices_to_augment.ndim != 1:
             logging.warning("Nieprawidłowy `indices_to_augment`. Pomijam augmentację.")
             return X_train.copy(), y_train.copy()
        if indices_to_augment.size == 0:
            logging.info("      - Brak indeksów do augmentacji.")
            return X_train.copy(), y_train.copy()

        logging.info(f"      - Rozpoczynam augmentację {len(indices_to_augment)} próbek ({n_augmentations_per_sample} aug/próbkę)...")
        X_aug_list, y_aug_list = [], []
        added_count = 0

        indices_iter = tqdm(indices_to_augment, desc="Augmentacja", leave=False, ncols=100) if len(indices_to_augment) > 500 else indices_to_augment

        for idx in indices_iter:
            if idx >= len(X_train): continue
            original_image = X_train[idx].astype(np.uint8)
            label = y_train[idx]
            for _ in range(n_augmentations_per_sample):
                # Łańcuch transformacji
                img = self._apply_affine(original_image)
                img = self._extract_digit(img)
                img = self._apply_morph(img)
                img = self._resize_center(img)
                if img is not None:
                    X_aug_list.append(img)
                    y_aug_list.append(label)
                    added_count += 1

        logging.info(f"      - Augmentacja zakończona. Dodano {added_count} obrazów.")

        if not X_aug_list:
             logging.info("      - Nie wygenerowano poprawnych augmentacji.")
             return X_train.copy(), y_train.copy()

        try: # Konkatenacja
            X_train_augmented = np.array(X_aug_list)
            y_train_augmented = np.array(y_aug_list)
            # Upewnij się, że X_train jest 3D
            if len(X_train.shape) == 2:
                side = int(np.sqrt(X_train.shape[1])); X_train = X_train.reshape(-1, side, side)

            if X_train.shape[1:] == X_train_augmented.shape[1:]:
                X_train_combined = np.concatenate((X_train, X_train_augmented), axis=0)
                y_train_combined = np.concatenate((y_train, y_train_augmented), axis=0)
            else: warnings.warn("..."); X_train_combined,y_train_combined=X_train.copy(),y_train.copy()
        except Exception as e_concat: logging.error(f"BŁĄD konkatenacji: {e_concat}"); X_train_combined,y_train_combined=X_train.copy(),y_train.copy()

        logging.info(f"      - Dane połączone. Nowy rozmiar X_train: {X_train_combined.shape}")
        return X_train_combined, y_train_combined

# --- ZOPTYMALIZOWANA funkcja create_pipeline ---
def create_pipeline(use_umap: bool, model_instance: BaseEstimator, **umap_params) -> Pipeline:
    """Tworzy pipeline z flatten, opcjonalnie UMAP i przekazanym modelem."""
    steps = [("flatten", flatten_transformer)]
    if use_umap:
        umap_params_filtered = {k: v for k, v in umap_params.items() if v is not None}
        umap_params_filtered.setdefault('random_state', RANDOM_STATE); umap_params_filtered.setdefault('n_jobs', 1)
        try: steps.append(("reducer", umap.UMAP(**umap_params_filtered)))
        except Exception as e: logging.warning(f"Błąd UMAP: {e}. Pomijam redukcję.");
    steps.append(("classifier", model_instance))
    return Pipeline(steps)

# --- ZREFAKTORYZOWANE Funkcje dla Objective ---
def _suggest_hyperparameters(trial: optuna.Trial, use_umap_flag: bool) -> Dict[str, Any]:
    """Sugeruje hiperparametry dla triala Optuny."""
    params = {
        'n_neighbors_lof': trial.suggest_int("n_neighbors_lof", 20, 300, step=10),
        'contamination_lof': trial.suggest_float("contamination_lof", 0.01, 0.25, step=0.01),
        'contamination_forest': trial.suggest_float("contamination_forest", 0.01, 0.25, step=0.01),
        'n_augmentations_per_sample': trial.suggest_int("n_augmentations_per_sample", 1, 3),
        'weak_threshold_factor': trial.suggest_float("weak_threshold_factor", 0.2, 3.0),
        'qda_reg_param': trial.suggest_float("qda_reg_param", 0.01, 0.3, step=0.05),
        'hgb_learning_rate': trial.suggest_float("hgb_learning_rate", 0.02, 0.15, log=True),
        'hgb_max_leaf_nodes': trial.suggest_int("hgb_max_leaf_nodes", 25, 50),
        'et_n_estimators': trial.suggest_int("et_n_estimators", 80, 250, step=10),
        'et_max_depth': trial.suggest_int("et_max_depth", 8, 25),
        'et_min_samples_split': trial.suggest_int("et_min_samples_split", 2, 10),
        'et_min_samples_leaf': trial.suggest_int("et_min_samples_leaf", 1, 5)
    }
    if use_umap_flag:
        params['n_components_umap'] = trial.suggest_int("n_components_umap", 15, 60, step=5)
        params['n_neighbors_umap'] = trial.suggest_int("n_neighbors_umap", 15, 300, step=15)
        params['min_dist'] = trial.suggest_float("min_dist", 0.05, 0.4)
        params['spread'] = trial.suggest_float("spread", 0.9, 1.1)
        # Stałe parametry UMAP można dodać w `create_pipeline`
    return params

def _build_classifier(params: Dict[str, Any]) -> BaseEstimator:
    """Buduje VotingClassifier na podstawie zasugerowanych parametrów."""
    try:
        qda_inst = QDA(reg_param=params['qda_reg_param'])
        hgb_inst = HistGradientBoostingClassifier(
            learning_rate=params['hgb_learning_rate'], max_leaf_nodes=params['hgb_max_leaf_nodes'],
            random_state=RANDOM_STATE)
        et_inst = ExtraTreesClassifier(
            n_estimators=params['et_n_estimators'], max_depth=params['et_max_depth'],
            min_samples_split=params['et_min_samples_split'],
            min_samples_leaf=params['et_min_samples_leaf'],
            random_state=RANDOM_STATE, n_jobs=1) # ET może używać wielu rdzeni
        current_estimators = [('qda', qda_inst), ('hgb', hgb_inst), ('et', et_inst)]
        # n_jobs=1 dla VotingClassifier dla stabilności, zwłaszcza przy n_jobs > 1 w Optunie
        voting_clf_inst = VotingClassifier(estimators=current_estimators, voting='soft', n_jobs=-1)
        return voting_clf_inst
    except Exception as e:
        logging.error(f"BŁĄD podczas budowania klasyfikatora: {e}", exc_info=True)
        raise # Rzuć wyjątek dalej

# --- ZOPTYMALIZOWANA i ZREFAKTORYZOWANA funkcja objective ---
def objective(
    trial: optuna.Trial, X_train_data: np.ndarray, y_train_data: np.ndarray,
    X_val_data: np.ndarray, y_val_data: np.ndarray, X_train_reduced_for_lof_data: np.ndarray,
    use_umap_flag: bool, evaluator_instance: ModelEvaluator, augmenter_instance: ImageAugmenter
) -> float:
    """Funkcja celu dla Optuna - zrefaktoryzowana."""
    trial_num = trial.number
    logging.info(f"--- Rozpoczęcie Trial {trial_num} (use_umap={use_umap_flag}) ---")
    score = 0.0

    try:
        # Krok 1: Sugerowanie hiperparametrów
        logging.info(f"  Trial {trial_num}: Sugerowanie hiperparametrów...")
        params = _suggest_hyperparameters(trial, use_umap_flag)
        logging.info(f"  Trial {trial_num}: Sugerowanie parametrów zakończone.")
        # Logowanie tylko kluczowych
        logging.info(f"  Trial {trial_num}: Kluczowe parametry: Aug(n={params['n_augmentations_per_sample']}, thr={params['weak_threshold_factor']:.2f}), "
                     f"LOF(c={params['contamination_lof']:.3f}), IF(c={params['contamination_forest']:.3f}), UMAP={use_umap_flag}")

        # Krok 2: Utworzenie modeli i pipeline
        model_instance = _build_classifier(params)
        umap_params = {k: v for k, v in params.items() if k.startswith('umap_') or k in ['n_components', 'n_neighbors', 'min_dist', 'spread']}
        pipeline = create_pipeline(use_umap=use_umap_flag, model_instance=model_instance, **umap_params)
        logging.info(f"  Trial {trial_num}: Pipeline utworzony.")

        # Krok 3: Identyfikacja próbek do augmentacji
        indices_to_augment = np.array([], dtype=int)
        try:
            weak_indices = find_weakly_described_indices(
                X_train_reduced_for_lof_data, y_train_data, threshold_factor=params['weak_threshold_factor']
            )
            outlier_indices = _detect_outliers(
                X_train_data, X_train_reduced_for_lof_data,
                params['n_neighbors_lof'], params['contamination_lof'], params['contamination_forest']
            )
            indices_to_augment = np.unique(np.concatenate((weak_indices, outlier_indices))).astype(int)
            logging.info(f"  Trial {trial_num}: Znaleziono {len(indices_to_augment)} unikalnych indeksów do augmentacji.")
        except Exception as e_detect: logging.error(f"BŁĄD identyfikacji indeksów: {e_detect}", exc_info=True)

        # Krok 4: Augmentacja Danych (użycie instancji Augmentera)
        logging.info(f"  Trial {trial_num}: Rozpoczynam augmentację danych...")
        X_train_aug, y_train_aug = X_train_data.copy(), y_train_data.copy() # Zacznij od kopii
        if indices_to_augment.size > 0:
            try:
                # Wywołaj metodę z instancji augmentera
                X_train_aug, y_train_aug = augmenter_instance.augment_batch(
                    X_train_data.copy(), y_train_data.copy(),
                    indices_to_augment, n_augmentations_per_sample=params['n_augmentations_per_sample']
                )
            except Exception as e_aug: logging.error(f"BŁĄD augmentacji: {e_aug}", exc_info=True); return 0.0
        logging.info(f"  Trial {trial_num}: Augmentacja zakończona. Rozmiar po: {len(y_train_aug)}")

        # Krok 5: Ewaluacja modelu
        logging.info(f"  Trial {trial_num}: Rozpoczynam ewaluację modelu...")
        eval_results = evaluator_instance.evaluate(
            pipeline=pipeline, X_train=X_train_aug, y_train=y_train_aug,
            X_val=X_val_data, y_val=y_val_data, add_flatten_step=False,
            calculate_proba=False, verbose=False
        )
        logging.info(f"  Trial {trial_num}: Ewaluacja zakończona.")

        # Krok 6: Obliczanie wyniku
        metrics_df = eval_results.get('metrics')
        if metrics_df is not None and not metrics_df.empty and not metrics_df.isnull().all().all(): # Sprawdź czy nie jest pusty/NaN
             req_metrics = ["accuracy", "precision", "recall", "f1"]
             valid_scores = [metrics_df[m].iloc[0] for m in req_metrics if m in metrics_df.columns and pd.notna(metrics_df[m].iloc[0])]
             if valid_scores: score = np.mean(valid_scores)
             else: logging.warning(f"Trial {trial_num}: Brak poprawnych metryk."); score = 0.0
        else: logging.warning(f"Trial {trial_num}: Brak 'metrics' DF lub same NaN."); score = 0.0
        logging.info(f"  Trial {trial_num}: Obliczony wynik = {score:.6f}")

        # Pruning (opcjonalne, ale dobre dla Optuny)
        trial.report(score, trial_num)
        if trial.should_prune():
             logging.info(f"Trial {trial_num} pruned.")
             raise optuna.TrialPruned()

    except optuna.TrialPruned: # Przechwyć i rzuć dalej
         raise
    except Exception as e_trial:
        logging.error(f"BŁĄD KRYTYCZNY w Trial {trial_num}: {e_trial}", exc_info=True)
        score = 0.0 

    # Zwalnianie pamięci
    vars_to_delete = ['X_train_aug', 'y_train_aug', 'pipeline', 'eval_results', 'model_instance',
                      'current_estimators', 'qda_inst', 'hgb_inst', 'et_inst', 'metrics_df',
                      'valid_scores', 'indices_to_augment', 'weak_indices', 'outlier_indices', 'params', 'umap_params']
    for var_name in vars_to_delete:
        if var_name in locals(): del locals()[var_name]
    gc.collect()
    return score

# --- Inicjalizacja Augmentera ---
augmenter = ImageAugmenter(config=AUGMENTATION_CONFIG, target_size=TARGET_IMG_SIZE)

# --- Optymalizacja: Podział danych RAZ ---
logging.info("Dzielenie upscalowanych danych na zbiory treningowe i walidacyjne...")
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    smoothed_X_train, y_train, test_size=0.2,
    stratify=y_train, random_state=RANDOM_STATE
)
logging.info(f"Rozmiary zbiorów: Trn={X_train_split.shape}/{y_train_split.shape}, Val={X_val_split.shape}/{y_val_split.shape}")
del smoothed_X_train, y_train; gc.collect()

# --- Optymalizacja: Pre-obliczenie UMAP dla LOF/Weak RAZ ---
logging.info(f"Pre-obliczanie UMAP ({PRE_UMAP_CONFIG['n_components']} komp.) dla LOF/Weak...")
X_train_flat_for_pre_umap = flatten(X_train_split) # Użyj funkcji flatten
pre_umap_reducer = umap.UMAP(**PRE_UMAP_CONFIG) # Użyj konfiguracji
try:
    X_train_reduced_pre_calc = pre_umap_reducer.fit_transform(X_train_flat_for_pre_umap)
    logging.info(f"UMAP embedding (LOF/Weak) obliczony. Kształt: {X_train_reduced_pre_calc.shape}")
    del X_train_flat_for_pre_umap; gc.collect()
except Exception as e_umap_pre: logging.error(f"BŁĄD UMAP dla LOF/Weak: {e_umap_pre}", exc_info=True); exit()

# --- Konfiguracja i uruchomienie Optuny ---
storage = optuna.storages.RDBStorage(OPTUNA_CONFIG['db_path'])
try:
    if OPTUNAHUB_AVAILABLE: sampler = optunahub.load_module(package="samplers/auto_sampler").AutoSampler()
    else: raise ImportError("Optunahub niezaładowany")
    logging.info("Używam AutoSampler z optunahub.")
except Exception: logging.warning(f"Używam TPESampler."); sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

# Dynamiczne nazwy studiów
study_name_umap = f"{OPTUNA_CONFIG['study_name_prefix_umap']}"
study_name_no_umap = f"{OPTUNA_CONFIG['study_name_prefix_no_umap']}"

# --- Wybór i uruchomienie JEDNEGO study na podstawie flagi ---
if RUN_WITH_UMAP_IN_PIPELINE:
    study_name = study_name_umap
    use_umap_objective_flag = True
    n_trials = OPTUNA_CONFIG['n_trials_umap']
    logging.info(f"--- Konfiguracja: Uruchamianie study Z UMAP w pipeline ('{study_name}') ---")
else:
    study_name = study_name_no_umap
    use_umap_objective_flag = False
    n_trials = OPTUNA_CONFIG['n_trials_no_umap']
    logging.info(f"--- Konfiguracja: Uruchamianie study BEZ UMAP w pipeline ('{study_name}') ---")

try:
    study = optuna.load_study(study_name=study_name, storage=storage, sampler=sampler)
    logging.info(f"Wczytano istniejące study '{study_name}' z {OPTUNA_CONFIG['db_path']}.")
except KeyError:
    study = optuna.create_study(direction="maximize", storage=storage, study_name=study_name, sampler=sampler)
    logging.info(f"Utworzono nowe study '{study_name}' w {OPTUNA_CONFIG['db_path']}.")

logging.info(f"--- Rozpoczynam optymalizację dla '{study_name}' ({n_trials} triali, {OPTUNA_CONFIG['n_jobs']} jobs) ---")
study.optimize(
    lambda trial: objective(
        trial,
        X_train_split, y_train_split, X_val_split, y_val_split,
        X_train_reduced_pre_calc,
        use_umap_flag=use_umap_objective_flag,
        evaluator_instance=evaluator,
        augmenter_instance=augmenter # Przekaż instancję augmentera
    ),
    n_trials=n_trials, n_jobs=OPTUNA_CONFIG['n_jobs'],
    gc_after_trial=True, callbacks=[callback]
)

# --- Wyświetlenie wyników dla uruchomionego study ---
print("\n" + "="*30 + " WYNIKI OPTYMALIZACJI " + "="*30)
print(f"\n--- Wyniki dla Study: {study.study_name} ---")
try:
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    if completed_trials:
        print(f"Liczba ukończonych triali: {len(completed_trials)}")
        print(f"Najlepsza wartość: {study.best_value:.6f}")
        print("Najlepsze parametry:")
        for key, value in study.best_params.items(): print(f"  {key}: {value}")
    else: print("Brak ukończonych triali dla tego study.")
except Exception as e: print(f"Błąd podczas wyświetlania wyników dla study '{study.study_name}': {e}")

print("\n" + "="*80)

In [ ]:
def augment_data(X_train, y_train, n_neighbors_lof, contamination_lof, 
                    contamination_forest):
    """
    Augmentuje dane treningowe poprzez:
        1. Wykrywanie outlierów za pomocą LOF i IsolationForest.
        2. Augmentację danych dla outlierów (przesunięcie, obrót, skalowanie, pochylenie).
        3. Redukcję wymiarów UMAP dla całego zestawu danych.
        4. Wizualizację outlierów i ich augmentacji.
    """

    # 1. Redukcja wymiarów UMAP dla LOF
    reducer = umap.UMAP(n_components=2)
    X_train_reduced = reducer.fit_transform(X_train.reshape(X_train.shape[0], -1))

    # 2. Wykrywanie outlierów (LOF)
    lof = LocalOutlierFactor(n_neighbors=n_neighbors_lof, contamination=contamination_lof)
    y_pred_lof = lof.fit_predict(X_train_reduced)

    # 3. Isolation Forest na surowych danych
    iso_forest = IsolationForest(contamination=contamination_forest)
    y_pred_iso = iso_forest.fit_predict(X_train.reshape(X_train.shape[0], -1))

    # 4. Połączenie wyników LOF i IsolationForest
    y_pred_combined = (y_pred_lof + y_pred_iso) / 2
    outlier_indices = np.where(y_pred_combined < 0)[0]
    print(f"Liczba outlinerów: {len(outlier_indices)}")
    
    
    # 5. Augmentacja danych dla outlierów
    X_train_augmented = []
    y_train_augmented = []

    if len(outlier_indices) > 0:
        n_outliers_to_show = min(12, len(outlier_indices))  # Ograniczenie liczby wyświetlanych outlierów
        fig, axes = plt.subplots(n_outliers_to_show, 5, figsize=(15, 3 * n_outliers_to_show))
        k = 1  # Zmienna do iterowania po kolumnach
        for i, idx in enumerate(outlier_indices[:n_outliers_to_show]):
            image = X_train[idx]
            label = y_train[idx]  # Pobranie etykiety dla outliera

            # 1. Binaryzacja oryginalnego obrazu
            _, binary_image = cv2.threshold(image.astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # 2. Znalezienie konturu cyfry
            contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                cnt = contours[0]

                # Oryginalny obraz
                axes[i, 0].imshow(image, cmap='gray')
                axes[i, 0].set_title(f'Outlier {i+1}')
                axes[i, 0].axis('off')

                # 1. Dodanie paddingu (czarne tło)
                padded_image = np.pad(image, pad_width=35, mode='constant')  # Zwiększony padding
                for j in range(4):  # Zmienna j zdefiniowana wewnątrz pętli
                    # 2. Transformacja afiniczna
                    transform = AffineTransform(scale=(np.random.uniform(0.97, 1.04), np.random.uniform(0.97, 1.04)),
                                                rotation=np.deg2rad(np.random.randint(-20, 21)),
                                                shear=np.deg2rad(np.random.randint(-5, 6))
                                                )  
                    transformed_image = warp(padded_image, transform, mode='wrap', preserve_range=True)
                    
                    # 3. Wyizolowanie cyfry po transformacji (Sobel)
                    grad_x = cv2.Sobel(transformed_image, cv2.CV_64F, 1, 0, ksize=3)
                    grad_y = cv2.Sobel(transformed_image, cv2.CV_64F, 0, 1, ksize=3)
                    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
                    grad_magnitude = cv2.normalize(grad_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                    _, thresh = cv2.threshold(grad_magnitude, 50, 255, cv2.THRESH_BINARY)

                    contours = find_contours(thresh, 0.75)

                    # --- Obsługa błędów ---
                    try:
                        cnt = contours[0]  # Wybór pierwszego konturu

                        # Wyznaczenie bounding box
                        minr = int(np.min(cnt[:, 0]))
                        minc = int(np.min(cnt[:, 1]))
                        maxr = int(np.max(cnt[:, 0]))
                        maxc = int(np.max(cnt[:, 1]))

                        # Sprawdzenie, czy bounding box ma poprawny kształt
                        if minr == maxr or minc == maxc:
                            print("Nieprawidłowy bounding box - pomijanie augmentacji dla tego obrazu.")
                            continue

                    except IndexError:
                        print("Nie znaleziono konturu - pomijanie augmentacji dla tego obrazu.")
                        continue

                    # 4. Wycięcie cyfry na podstawie bounding box
                    isolated_digit = transformed_image[minr:maxr, minc:maxc]
                
                    # --- Erozja ---
                    if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                        isolated_digit = erosion(isolated_digit, disk(1))
                        
                    # --- Otwarcie  ---
                    if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                        isolated_digit = opening(isolated_digit, disk(1))
                        
                    # --- Zamknięcie 
                    if np.random.rand() < 0.5:  # Zastosuj z prawdopodobieństwem 50%
                        isolated_digit = closing(isolated_digit, disk(1))
                                    
                    
                    # 5. Wycentrowanie cyfry w obrazie 42x42
                    new_image = np.zeros((42, 42), dtype=np.uint8)
                    row_offset = max(0, (42 - (maxr - minr)) // 2)
                    col_offset = max(0, (42 - (maxc - minc)) // 2)
                    new_image[row_offset:row_offset + maxr - minr, col_offset:col_offset + maxc - minc] = isolated_digit

                    isolated_digit = new_image

                    # 6. Dodanie do listy
                    X_train_augmented.append(isolated_digit)  
                    y_train_augmented.append(label)  


                    # Wyświetlenie obrazu
                    axes[i, k].imshow(new_image, cmap='gray')
                    axes[i, k].axis('off')
                    k += 1  # Przejście do następnej kolumny
                    if k == 5:  # Jeśli osiągnięto ostatnią kolumnę
                        k = 1  # Resetuj k do pierwszej kolumny dla następnego outliera
            else:
                print("Nie znaleziono konturu - pomijanie augmentacji dla tego obrazu.")

        plt.tight_layout()
        plt.show()

    X_train_augmented = np.array(X_train_augmented)
    y_train_augmented = np.array(y_train_augmented)

    # Połączenie danych augmentowanych z oryginalnymi danymi
    X_train_augmented = np.concatenate((X_train, X_train_augmented), axis=0)
    y_train_augmented = np.concatenate((y_train, y_train_augmented), axis=0)

    return X_train_augmented, y_train_augmented, outlier_indices, X_train_reduced


X_train_Upsc, X_val_Upsc, y_train_Upsc, y_val_Upsc = train_test_split(smoothed_X_train, y_train, test_size=0.2, random_state=69)

# Wywołanie funkcji augment_data
X_train_augmented, y_train_augmented, outlier_indices, X_train_reduced = augment_data(
    X_train_Upsc,
    y_train_Upsc,
    n_neighbors_lof=399,
    contamination_lof=0.33,
    contamination_forest=0.005
)

### EfficientNetV2 

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L 
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


def create_efficientnetv2_model(
    input_shape,
    num_classes,
    efficientnet_version="B0",
    trainable=False,
    dropout_rate=0.2,
    weights=None
):
    """Tworzy model oparty na EfficientNetV2.

    Args:
        input_shape: Dane wejściowe.
        num_classes: Liczba klas.
        efficientnet_version: Wersja EfficientNetV2 ('B0', 'S', 'M', 'L').
        trainable: Czy warstwy EfficientNetV2 mają być trenowalne.
        dropout_rate: Współczynnik dropout.

    Returns:
        Model Keras.
    """

    # Wybór wersji EfficientNetV2
    if efficientnet_version == "B0":
        base_model = EfficientNetV2B0(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "S":
        base_model = EfficientNetV2S(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "M":
        base_model = EfficientNetV2M(
            include_top=False, weights=weights, input_shape=input_shape
        )
    elif efficientnet_version == "L":
        base_model = EfficientNetV2L(
            include_top=False, weights=weights, input_shape=input_shape
        )
    else:
        raise ValueError("Nieprawidłowa wersja EfficientNetV2.")

    # Ustawienie warstw EfficientNetV2 jako trenowalne
    base_model.trainable = trainable

    # Budowa modelu
    inputs = Input(shape=input_shape)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

# Baseline data without transformations and agumentation 
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(smoothed_X_train, y_train_ENC, test_size=0.2, random_state=69)

#{'n_neighbors_lof': 222, 'contamination_lof': 0.1625, 'contamination_forest': 0.195}.
# Odtworzenie augmentacji z najlepszymi parametrami
X_train_augmented, y_train_augmented, _, _ = augment_data(
    X_train_base, 
    y_train_base, 
    n_neighbors_lof=222,
    contamination_lof=0.1625,
    contamination_forest=0.195
)

X_train_base = np.expand_dims(X_train_base, axis=-1)
X_val_base = np.expand_dims(X_val_base, axis=-1)  
X_train_augmented = np.expand_dims(X_train_augmented, axis=-1)

X_test = np.squeeze(X_test) 
X_test = np.expand_dims(X_test, axis=-1)

print("Shape of X_train_base:", X_train_base.shape)
print("Shape of y_train_base:", y_train_base.shape)

print("Shape of X_val_base:", X_val_base.shape)
print("Shape of y_val_base:", y_val_base.shape)

print("Shape of X_train_augmented:", X_train_augmented.shape)
print("Shape of y_train_augmented:", y_train_augmented.shape)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test_ENC.shape)

# --- Trenowanie modeli ---
# Model na danych bez redukcji
model_base = create_efficientnetv2_model(
    input_shape=X_train_base.shape[1:], 
    num_classes=10, 
    efficientnet_version="S",
    trainable=True,
    weights=None,
)

model_base.compile(
    optimizer="Lion", loss="categorical_crossentropy", metrics=["categorical_accuracy"] 
)

history_base = model_base.fit(
    X_train_base,
    y_train_base,
    epochs=36,
    batch_size=512,
    validation_data=(X_val_base, y_val_base),
)

# Model na danych z redukcją
model_transformed = create_efficientnetv2_model(
    input_shape=X_train_augmented.shape[1:],  
    num_classes=10, 
    efficientnet_version="S",
    trainable=True,
    weights=None,
)

model_transformed.compile(
    optimizer="Lion", loss="categorical_crossentropy", metrics=["categorical_accuracy"] 
)

history_transformed = model_transformed.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=36,
    batch_size=512,
    validation_data=(X_val_base, y_val_base),
)

# --- Predykcja ---
y_pred_base = model_base.predict(X_test)
y_pred_transformed = model_transformed.predict(X_test)

# --- Porównanie wyników ---
def calculate_metrics(y_true, y_pred):
    """Oblicza metryki."""
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    return {
        "accuracy": accuracy_score(y_true_labels, y_pred_labels),
        "precision": precision_score(y_true_labels, y_pred_labels, average="macro"),
        "recall": recall_score(y_true_labels, y_pred_labels, average="macro"),
        "f1": f1_score(y_true_labels, y_pred_labels, average="macro"),
    }

metrics_base = calculate_metrics(y_test_ENC, y_pred_base)
metrics_transformed = calculate_metrics(y_test_ENC, y_pred_transformed)

print("Metryki dla modelu bez agumentacji:", metrics_base)
print("Metryki dla modelu z agumentacją:", metrics_transformed)

## Model B0
# Dla danych Walidacyjnych (Pochodzące z zbiru danych Treningowych)
# Metryki dla modelu bez agumentacji: {'accuracy': 0.9922,  'precision': 0.9921916598777563, 'recall': 0.9922927046235455, 'f1': 0.992236418206011}
# Metryki dla modelu z agumentacją:   {'accuracy': 0.9895,  'precision': 0.9896354567051183, 'recall': 0.9895893078591114, 'f1': 0.9895911709150041}

# Metryki dla danych testowych
#  Metryki dla modelu bez agumentacji: {'accuracy': 0.9816, 'precision': 0.9814288255960995, 'recall': 0.9815351078380796, 'f1': 0.9814747112782948}
#  Metryki dla modelu z agumentacją:   {'accuracy': 0.9896, 'precision': 0.9896281452846342, 'recall': 0.9894803578385878, 'f1': 0.9895399474757763}

## Model S
# Metryki dla danych testowych
# Metryki dla modelu bez agumentacji: {'accuracy': 0.9925, 'precision': 0.9924837719191659, 'recall': 0.9924530861244054, 'f1': 0.992463415535193}
# Metryki dla modelu z agumentacją:   {'accuracy': 0.9921, 'precision': 0.9920614949658724, 'recall': 0.9920511460346548, 'f1': 0.9920482305787767}

# Deklaracja funkcji dla ekstrakcji cech z obrazu

In [ ]:

def calculate_hog_parallel(images, orientations=None, pixels_per_cell=None, cells_per_block=None, n_jobs=-1):
    """Oblicza deskryptory HOG dla obrazu."""
    
     # Ustawienie wartości domyślnych, jeśli parametry nie są podane
    if orientations is None:
        orientations = 9
    if pixels_per_cell is None:
        pixels_per_cell = (8, 8)
    if cells_per_block is None:
        cells_per_block = (2, 2)
    
    def calculate_hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block):
        
        image = img_as_float32(image)

        fd = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block, visualize=False, channel_axis=None)

        return fd
    
    return Parallel(n_jobs=n_jobs)(delayed(calculate_hog)(image, orientations, pixels_per_cell, cells_per_block) for image in images)


def apply_gabor_filters_parallel(images, thetas=None, sigmas=None, frequencies=None, n_jobs=-1):
    """
    Stosuje filtry Gabora do wielu obrazów równolegle.

    Argumenty:
        images: Lista obrazów wejściowych.
        thetas: (Opcjonalnie) Lista kątów orientacji filtrów Gabora (w radianach). Jeśli None, używane są wartości domyślne.
        sigmas: (Opcjonalnie) Lista odchyleń standardowych filtrów Gabora. Jeśli None, używane są wartości domyślne.
        frequencies: (Opcjonalnie) Lista częstotliwości przestrzennych filtrów Gabora. Jeśli None, używane są wartości domyślne.
        n_jobs: Liczba rdzeni procesora do wykorzystania podczas zrównoleglania. -1 oznacza użycie wszystkich dostępnych rdzeni.

    Zwraca:
        Listę tablic NumPy zawierających obrazy przefiltrowane filtrami Gabora, o typie danych float32.
    """

    # Ustawienie wartości domyślnych, jeśli parametry nie są podane
    if thetas is None:
        thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    if sigmas is None:
        sigmas = [1, 3]
    if frequencies is None:
        frequencies = [0.05, 0.25]

    def apply_gabor_filters_single(image):
        image = img_as_float32(image)
        filtered_images = [gabor(image, frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)[0]
                           for theta in thetas for sigma in sigmas for frequency in frequencies]
        return np.array(filtered_images, dtype=np.float32)

    return Parallel(n_jobs=n_jobs)(delayed(apply_gabor_filters_single)(image) for image in images)


def extract_features_HOG_Gabor(X, hog_orientations=None, hog_pixels_per_cell=None, hog_cells_per_block=None, 
                              gabor_thetas=None, gabor_sigmas=None, gabor_frequencies=None):
    """Generator zwracający cechy HOG i Gabora dla obrazów."""

    hog_features_list = calculate_hog_parallel(X, orientations=hog_orientations, pixels_per_cell=hog_pixels_per_cell, cells_per_block=hog_cells_per_block)
    gabor_features_list = apply_gabor_filters_parallel(X, thetas=gabor_thetas, sigmas=gabor_sigmas, frequencies=gabor_frequencies)

    for hog_features, gabor_features in zip(hog_features_list, gabor_features_list):
        
        gabor_features_flattened = np.array([img.flatten() for img in gabor_features])

        # Dopasowujemy kształt hog_features do gabor_features_flattened
        hog_features_reshaped = np.repeat(hog_features.reshape(1, -1), gabor_features_flattened.shape[0], axis=0)

        # Łączymy cechy HOG i Gabora w poziomie
        concatenated_features = np.hstack((hog_features_reshaped, gabor_features_flattened))

        concatenated_features = concatenated_features.flatten()

        yield concatenated_features

#################################################################################################################################################
################################################  Directional filters, ORB    ###################################################################

def apply_directional_filters(image, kernel_sizes=[3, 5]):
    """Stosuje filtry kierunkowe do obrazu."""
    filtered_images = []

    # Wykrywanie krawędzi filtrem Canny'ego
    edges = cv2.Canny(image, 50, 150)

    def process_kernel(kernel_size):
        if kernel_size % 2 == 0 or kernel_size > 31:
            raise ValueError(
                f"Nieprawidłowy rozmiar jądra: {kernel_size}. Rozmiar jądra musi być nieparzysty i nie większy niż 31."
            )
        kernel_horizontal = cv2.getDerivKernels(1, 0, kernel_size,
                                               normalize=True)
        kernel_vertical = cv2.getDerivKernels(0, 1, kernel_size,
                                             normalize=True)
        filtered = cv2.sepFilter2D(edges, cv2.CV_32F, kernel_horizontal[0],
                                  kernel_vertical[0])
        return filtered.flatten()

    with Pool() as pool:
        filtered_images = pool.map(process_kernel, kernel_sizes)

    # Analiza składowych spójnych
    labels = label(edges)
    props = regionprops(labels)
    features = [prop.area for prop in props]

    return np.concatenate(
        [np.array(filtered_images).squeeze(), np.array(features)])


def extract_orb_features(image, nfeatures=100):
    """Wyodrębnia cechy ORB z obrazu."""
    orb = cv2.ORB_create(nfeatures=nfeatures,
                         fastThreshold=12,
                         edgeThreshold=12)
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return []
    return descriptors.flatten()


def extract_combined_features(image, kernel_sizes=[3, 5], nfeatures=100):
    """Łączy cechy z filtrów kierunkowych i ORB."""
    directional_features = apply_directional_filters(image, kernel_sizes)
    orb_features = extract_orb_features(image, nfeatures)

    # Łączenie cech
    combined_features = np.concatenate((directional_features, orb_features))

    return combined_features
   
#################################################################################################################################################
################################################  GLCM, Zernike    ##############################################################################
def extract_glcm_features(image, distances=[1], angles=[0], properties=['contrast', 'energy', 'homogeneity', 'correlation']):
    """Wyodrębnia cechy GLCM z obrazu."""
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    glcm = graycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)

    features = []
    for prop in properties:
        features.extend(graycoprops(glcm, prop).flatten())

    return np.array(features)


## OPTUNA dla HOG, Gabor, PCA

In [ ]:
# import warnings
# warnings.filterwarnings("ignore", module="optuna")


# def objective(trial, X_train, y_train):
#     """
#     Funkcja celu dla optymalizacji Optuna. 
#     Przyjmuje próbę (trial), dane treningowe (X_train) i etykiety (y_train).
#     Zwraca dokładność QDA i SVM na danych walidacyjnych po zastosowaniu sekwencji transformacji.
#     """

#     # Parametry HOG
#     orientations      = trial.suggest_int("hog_orientations", 2, 16)
#     pixels_per_cell   = trial.suggest_categorical("hog_pixels_per_cell", [str((2, 2)), str((2, 3)), str((3, 2)), str((3, 3)), str((4, 4)), str((5, 5)),str((6, 6)), str((7, 7))])
#     cells_per_block   = trial.suggest_categorical("hog_cells_per_block", [str((1, 1)), str((2, 2)), str((3, 3)), str((4, 4))])

#     # Parametry filtrów Gabora
#     gabor_thetas      = trial.suggest_categorical("gabor_thetas", ([0],  [np.pi / 14],  [np.pi / 12], [0, np.pi / 14 ], [0, np.pi / 12 ] ))
#     gabor_sigmas      = trial.suggest_categorical("gabor_sigmas", ([0.33], [0.66], [1], [0.33 , 1.25], [0.66, 2.35], [1, 3]))
#     gabor_frequencies = trial.suggest_categorical("gabor_frequencies", ([0.025], [0.05], [0.125], [0.025, 0.125], [0.05, 0.175], [0.05, 0.25]))
        
#     #PCA
#     PCA_n_components  =  trial.suggest_int("PCA_Components", 4, 228)
#     kernel = trial.suggest_categorical("kernel_pca_kernel", ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])
#     gamma = None
#     if kernel in ['poly', 'rbf']:
#         gamma = trial.suggest_loguniform("kernel_pca_gamma", 0.001, 1)
        
#     # Powiększanie i wygładzanie
#     # X_train_split = np.array([enlarge_and_smooth(img, new_size) for img in X_train])
#     # X_val = np.array([enlarge_and_smooth(img, new_size) for img in X_val])
    
#     # Ekstrakcja cech HOG i Gabor
#     X_train_features = np.array(list(extract_features_HOG_Gabor(
#         X_train, 
#         hog_orientations=orientations, 
#         hog_pixels_per_cell=eval(pixels_per_cell), # Konwersja stringa na krotkę
#         hog_cells_per_block=eval(cells_per_block),  # Konwersja stringa na krotkę
#         gabor_thetas=gabor_thetas, 
#         gabor_sigmas=gabor_sigmas, 
#         gabor_frequencies=gabor_frequencies
#     )))

#     # Podział na zbiór treningowy i walidacyjny
#     X_train_filters, X_val_filters, y_train_filters, y_val_filters = train_test_split(X_train_features, y_train, test_size=0.2, random_state=69)
    
    
#     # Kernel PCA
#     kpca = KernelPCA(n_components=PCA_n_components, kernel=kernel, gamma=gamma)
#     X_train_reduced = kpca.fit_transform(X_train_filters)
#     X_val_reduced = kpca.transform(X_val_filters)
                
#     # SVC
#     svc = SVC(kernel='rbf')
#     svc.fit(X_train_reduced, y_train_filters)
#     y_pred_svm = svc.predict(X_val_reduced)
#     accuracy_svm = accuracy_score(y_val_filters, y_pred_svm)

#     return accuracy_svm

# # Optymalizacji
                            
# study = optuna.create_study(directions=["maximize"], sampler=optuna.samplers.CmaEsSampler() ) 
# study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, n_jobs=1)


# print("Best parameters:", study.best_params)
# print("Best QDA accuracy:", study.best_trial.values[0]) 
# print("Best SVM accuracy:", study.best_trial.values[1]) 

# # [I 2024-09-17 05:43:37,964] Trial 13 finished with value: 0.99133 and parameters: {'hog_orientations': 16, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(3, 3)', 'gabor_thetas': [0.2243994752564138], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.025], 'PCA_Components': 209, 'kernel_pca_kernel': 'cosine'}. Best is trial 13 with value: 0.9913333333333333.
# # [I 2024-09-17 08:45:58,345] Trial 19 finished with value: 0.99175 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(2, 3)', 'hog_cells_per_block': '(2, 2)', 'gabor_thetas': [0.2617993877991494], 'gabor_sigmas': [1], 'gabor_frequencies': [0.025], 'PCA_Components': 132, 'kernel_pca_kernel': 'sigmoid'}. Best is trial 19 with value: 0.99175.
# # [I 2024-09-17 10:45:29,605] Trial 23 finished with value: 0.99225 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(3, 3)', 'hog_cells_per_block': '(2, 2)', 'gabor_thetas': [0], 'gabor_sigmas': [1], 'gabor_frequencies': [0.05, 0.25], 'PCA_Components': 150, 'kernel_pca_kernel': 'cosine'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 13:24:59,082] Trial 28 finished with value: 0.99108 and parameters: {'hog_orientations': 9, 'hog_pixels_per_cell': '(3, 2)', 'hog_cells_per_block': '(1, 1)', 'gabor_thetas': [0, 0.2617993877991494], 'gabor_sigmas': [1], 'gabor_frequencies': [0.05, 0.25], 'PCA_Components': 190, 'kernel_pca_kernel': 'sigmoid'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 13:54:44,248] Trial 29 finished with value: 0.9915 and parameters: {'hog_orientations': 7, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0.2617993877991494], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.05], 'PCA_Components': 164, 'kernel_pca_kernel': 'cosine'}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 18:02:03,444] Trial 37 finished with value: 0.99066 and parameters: {'hog_orientations': 9, 'hog_pixels_per_cell': '(3, 2)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0], 'gabor_sigmas': [0.66, 2.35], 'gabor_frequencies': [0.05, 0.175], 'PCA_Components': 169, 'kernel_pca_kernel': 'rbf', 'kernel_pca_gamma': 0.0032186977490984547}. Best is trial 23 with value: 0.99225.
# # [I 2024-09-17 19:02:58,946] Trial 39 finished with value: 0.9915 and parameters: {'hog_orientations': 6, 'hog_pixels_per_cell': '(4, 4)', 'hog_cells_per_block': '(4, 4)', 'gabor_thetas': [0, 0.2243994752564138], 'gabor_sigmas': [1, 3], 'gabor_frequencies': [0.125], 'PCA_Components': 148, 'kernel_pca_kernel': 'linear'}. Best is trial 23 with value: 0.99225.


## Wykorzystanie wcześniej otrzymanch hiperparamatrów dla HOG i filtrów Gabora po KPCA w celu stworzenia wektora cech z obrazów.

In [ ]:
# X_train_features = np.array(list(extract_features_HOG_Gabor(
#         X_train, 
#         hog_orientations=6, 
#         hog_pixels_per_cell=eval('(3, 3)'), 
#         hog_cells_per_block=eval('(2, 2)'),  
#         gabor_thetas=[0], 
#         gabor_sigmas=[1], 
#         gabor_frequencies=[0.05, 0.25]
#     )))


# kpca = KernelPCA(n_components=150, kernel='cosine')

# X_train_filters, X_val_filters, y_train_filters, y_val_filters = train_test_split(X_train_features, y_train_ENC, test_size=0.2, random_state=69)

# X_train_reduced = kpca.fit_transform(X_train_filters)
# X_val_reduced = kpca.transform(X_val_filters)


# np.save('X_train_reduced.npy', X_train_reduced)
# np.save('X_val_reduced.npy', X_val_reduced)
# np.save('y_train_filters.npy', y_train_filters)
# np.save('y_val_filters.npy', y_val_filters)


#### Pierwszy model na sieci konwolucyjnej dla bazy z HOG i Gabora.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LayerNormalization , BatchNormalization, PReLU , Input, ActivityRegularization, GlobalAveragePooling1D, SeparableConv1D 
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Lion
from tensorflow.keras.callbacks import EarlyStopping
import gc



X_train_reduced = np.load('X_train_reduced.npy')
X_val_reduced = np.load('X_val_reduced.npy')
y_train_filters = np.load('y_train_filters.npy')
y_val_filters = np.load('y_val_filters.npy')

X_train_reduced = X_train_reduced.reshape(X_train_reduced.shape[0], X_train_reduced.shape[1], 1)
X_val_reduced = X_val_reduced.reshape(X_val_reduced.shape[0], X_val_reduced.shape[1], 1)

def calculate_max_pool_size(input_length, prev_pool_size, min_output_size=2):
    calculated_pool_size = input_length // prev_pool_size
    return max(min_output_size, calculated_pool_size)


def objective(trial):
    """Funkcja celu dla Optuny."""
    #os.environ['TF_NUM_INTRAOP_THREADS'] = '1' ## Czy Kers działa na pojedycznym wątku ????
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()

    activation_functions = ['relu', 'elu', 'leaky_relu', 'tanh']
    activation = trial.suggest_categorical('activation', activation_functions)

    # Definiowanie modelu
    model = Sequential()
    model.add(Input(shape=(X_train_reduced.shape[1], 1)))

    # Pierwsza warstwa konwolucyjna
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_1', 48, 450),
                     kernel_size=trial.suggest_int('kernel_size_1', 2, 16),
                     #activation=activation,
                     padding='same', 
                     #dilation_rate=trial.suggest_int('dilation_rate_1', 1, 4)
                     )
              )
    model.add(PReLU())                #$$$$$$$ LAYER przed PRelu
    model.add(LayerNormalization())   ######
    model.add(ActivityRegularization(l1=trial.suggest_float('l1_1', 1e-6, 1e-1, log=True), 
                                     l2=trial.suggest_float('l2_1', 1e-6, 1e-1, log=True)
                                     )
              )
    pool_size_1 = trial.suggest_int('pool_size_1', 2, 10)
    model.add(MaxPooling1D(pool_size=pool_size_1))



    # Druga warstwa konwolucyjna
    input_length_2 = X_train_reduced.shape[1] // pool_size_1
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_2', 24, 400),
                     kernel_size=trial.suggest_int('kernel_size_2', 2, 14),
                     padding='same',
                     #activation=activation,
                     #dilation_rate=trial.suggest_int('dilation_rate_2', 1, 4)
                     )
              )
    model.add(PReLU())
    model.add(LayerNormalization())
    model.add(ActivityRegularization(l1=trial.suggest_float('l1_2', 1e-6, 1e-1, log=True), 
                                     l2=trial.suggest_float('l2_2', 1e-6, 1e-1, log=True)
                                     )
              )
    pool_size_2 = trial.suggest_int('pool_size_2', 2, input_length_2 // 2)
    #pool_size_2 = trial.suggest_int('pool_size_2', 2, 10)
    model.add(MaxPooling1D(pool_size=pool_size_2))



    # Trzecia warstwa konwolucyjna
    #input_length_3 = input_length_2 // pool_size_2
    model.add(Conv1D(filters=trial.suggest_int('conv1d_filters_3', 32, 300),
                     kernel_size=trial.suggest_int('kernel_size_3', 2, 12),
                     padding='same',
                     )
              )
    model.add(PReLU())
    model.add(LayerNormalization())
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_3', 0.01, 0.5)
                      )
              )
    #pool_size_3 = trial.suggest_int('pool_size_3', 2, max(2, input_length_3 // 2))
    #pool_size_3 = trial.suggest_int('pool_size_3', 2, 8)
    #model.add(MaxPooling1D(pool_size=pool_size_3))

    # Warstwy Dense
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(units=trial.suggest_int('dense_units_1', 40, 300), activation=activation))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_4', 0.01, 0.75)))
    
    model.add(Dense(units=trial.suggest_int('dense_units_2', 20, 200), activation=activation))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate_5', 0.01, 0.65)))
    
    model.add(Dense(10, activation='softmax'))

    # Optymalizator
    lr_initial = trial.suggest_float("lr_initial", 1e-5, 1e-1, log=True)
    decay_steps = trial.suggest_int("decay_steps", 10, 150)
    decay_rate = trial.suggest_float("decay_rate", 0.01, 0.99)
    lr_schedule = ExponentialDecay(initial_learning_rate=lr_initial,
                                  decay_steps=decay_steps,
                                  decay_rate=decay_rate)

    optimizer_name = trial.suggest_categorical('optimizer', ['Lion'])
    optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    optimizer = optimizer_class(learning_rate=lr_schedule)

    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=16, start_from_epoch=10, min_delta = 0.00005)

    # Trenowanie modelu
    model.fit(X_train_reduced, y_train_filters,
              epochs=trial.suggest_int('epochs', 40, 150),
              batch_size=trial.suggest_int('batch_size', 64, 1920),
              validation_data=(X_val_reduced, y_val_filters),
              verbose=1,
              callbacks=[early_stopping])

    # Ewaluacja modelu
    _, accuracy = model.evaluate(X_val_reduced, y_val_filters, verbose=0)

    del model
    return accuracy

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) ## Czy keras działa na wszystkich wątkach, ale optuna na pojedynczym. 


# Najlepsze parametry i wynik
print("Najlepsze parametry:", study.best_params)
print("Najlepsza dokładność:", study.best_value)

#[I 2024-10-04 05:18:28,377] Trial 26 finished with value: 0.9810000061988831 and parameters: {'activation': 'elu', 'conv1d_filters_1': 347, 'kernel_size_1': 7,  'l1_1': 0.0002141584459916536, 'l2_1': 4.201061472934882e-06, 'pool_size_1': 6, 'conv1d_filters_2': 77, 'kernel_size_2': 14, 'l1_2': 3.203529032832527e-06, 'l2_2': 0.00015793896479891322, 'pool_size_2': 2, 'conv1d_filters_3': 258, 'kernel_size_3': 4, 'dropout_rate_3': 0.3297488074504043, 'dense_units_1': 271, 'dropout_rate_4': 0.5097723961373417, 'dense_units_2': 78, 'dropout_rate_5': 0.15473996425601394, 'lr_initial': 0.0004175922071628326, 'decay_steps': 33, 'decay_rate': 0.9780884395210858, 'optimizer': 'Lion', 'epochs': 123, 'batch_size': 264}. Best is trial 26 with value: 0.9810000061988831.
#[I 2024-10-04 14:58:14,236] Trial 61 finished with value: 0.9797499775886536 and parameters: {'activation': 'elu', 'conv1d_filters_1': 424, 'kernel_size_1': 13, 'l1_1': 0.0005501543495144062, 'l2_1': 0.02794377888054429,   'pool_size_1': 7, 'conv1d_filters_2': 79, 'kernel_size_2': 12, 'l1_2': 1.3699264627909325e-06, 'l2_2': 6.577814668979489e-06, 'pool_size_2': 3, 'conv1d_filters_3': 280, 'kernel_size_3': 3, 'dropout_rate_3': 0.21341586312675653, 'dense_units_1': 292, 'dropout_rate_4': 0.22415600475665626, 'dense_units_2': 106, 'dropout_rate_5': 0.09686035013603865, 'lr_initial': 0.0004430745906851504, 'decay_steps': 22, 'decay_rate': 0.9872365423551039, 'optimizer': 'Lion', 'epochs': 150, 'batch_size': 216}. Best is trial 26 with value: 0.9810000061988831.


## Sieć CONV2D z zamrożonym PCA dla danych pierwotnych

In [ ]:
# import tensorflow as tf
# import keras
# from keras import layers
# import tensorflow_probability as tfp
# import optuna
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
# X_train_image = X_train_image.astype('float32')
# X_val_image = X_val_image.astype('float32')
# X_train_image = np.expand_dims(X_train_image, axis=-1)  
# X_val_image = np.expand_dims(X_val_image, axis=-1) 
 
# def objective(trial):
#     # Przestrzeń hiperparametrów
#     num_filters1 = trial.suggest_int("num_filters1", 32, 256)    # Pierwsza warstwa Conv2d
#     num_filters2 = trial.suggest_int("num_filters2", 64, 256)    # Druga warstwa Conv2d
#     num_filters3 = trial.suggest_int("num_filters3", 128, 256)   # Trzecia warstwa Conv2d
#     n_components = trial.suggest_int("PCA_components", 350, 500) # PCA

#     l1_1=trial.suggest_float('l1_1', 1e-6, 1e-1, log=True)
#     l2_1=trial.suggest_float('l2_1', 1e-6, 1e-1, log=True)
    
#     l1_2=trial.suggest_float('l1_2', 1e-6, 1e-1, log=True)
#     l2_2=trial.suggest_float('l2_2', 1e-6, 1e-1, log=True)

#     l1_3=trial.suggest_float('l1_3', 1e-6, 1e-1, log=True)
#     l2_3=trial.suggest_float('l2_3', 1e-6, 1e-1, log=True)
    
#     activation_functions = ['relu', 'gelu', 'leaky_relu', 'silu', 'mish']
#     activation = trial.suggest_categorical('activation', activation_functions)
    
    
#     # Warstwy konwolucyjne 2D
#     conv_layers = [
#         # Pierwsza warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters1,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_1),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_1),
#                                 padding='same',
#                                 input_shape=(X_train_image.shape[1:])
#                                 ),
        
#         tf.keras.layers.MaxPool2D(pool_size=2,strides=1,
#                                  ),
#         # Druga warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters2,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_2),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_2),                                
#                                 padding='same',
#                                 ),
#         tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
#         # Trzecia warstwa Conv2d
#         tf.keras.layers.Conv2D(
#                                 num_filters3,
#                                 kernel_size=2,
#                                 activation=activation,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l2_3),
#                                 bias_regularizer=tf.keras.regularizers.l1(l1_3),                                
#                                 padding='same'
#                                 ),
#         tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
#         tf.keras.layers.Flatten(),
#                 ]

#     ### Obliczenia dla zamrożonego PCA na podstawie pierwszej inicjalizacji modelu
#     conv_model = keras.models.Sequential(conv_layers)  
#     X_train_PCA = conv_model.predict(X_train_image)
    
#     X_train_PCA = tf.reshape(X_train_PCA, [X_train_PCA.shape[0], -1])

#     # Macierz kowariancji
#     covariance_matrix = tfp.stats.covariance(X_train_PCA, sample_axis=0, event_axis=-1)
#     # Wektory i wartości własne
#     eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
#     sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
#     eigenvalues = tf.gather(eigenvalues, sorted_indices)
#     eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
    
#     #### Opcja druga na SVD, ale wymaga jeszcze zabawy
#     # s, u, v = tf.linalg.svd(covariance_matrix)
#     # eigenvectors = v
#     # eigenvalues = tf.square(s) 
    
#     # Warstwa PCA
#     model = keras.models.Sequential(conv_layers + [
#         tf.keras.layers.Lambda(lambda x, eigenvectors: tf.matmul(x, eigenvectors), arguments={'eigenvectors': eigenvectors[:, :n_components]})
#     ])

#     # Output (klasyfikacja)
#     model.add(tf.keras.layers.Dense(10, activation='softmax'))
#     #model.summary()
#     tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    

#     # Optymalizator
#     optimizer_name = trial.suggest_categorical('optimizer', ['AdamW'])
#     optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
#     # Początkową wartość learning rate
#     lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
#     optimizer = optimizer_class(learning_rate=lr_initial)  

#     ## Kompilacja modelu
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     # Early Stopping
#     early_stopping = EarlyStopping(monitor='categorical_accuracy', patience=8, start_from_epoch=8, min_delta=0.0001)
    
#     # ReduceLROnPlateau
#     reduce_lr = ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.9, step=0.1),
#         patience=trial.suggest_int("reduce_lr_patience", 2, 12),
#         min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-4, log=True)
#     )
    
#     # Trenowanie i ocena modelu
#     history = model.fit(X_train_image, y_train_image,
#                         epochs=trial.suggest_int('epochs', 15, 80),
#                         batch_size=trial.suggest_int('batch_size', 64, 1920),
#                         validation_data=(X_val_image, y_val_image),
#                         callbacks=[early_stopping, reduce_lr]
#                         )
    
#     accuracy = history.history['val_categorical_accuracy'][-1]

#     return accuracy


# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) 

# # Najlepsze parametry i wynik
# print("Najlepsze parametry:", study.best_params)
# print("Najlepsza dokładność:", study.best_value)

# ### dwa tygodnie więc lepiej aby te wartości stanowiły dobrą podstawę...
# #Trial 76 finished with value:  0.9906666874885559 and parameters: {'num_filters1': 194, 'num_filters2': 167, 'num_filters3': 221, 'PCA_components': 383, 'l1_1': 1.4917571854023011e-06, 'l2_1': 0.0003844462705458024,  'l1_2': 0.0011921068803372,     'l2_2': 0.00021948428596782946, 'l1_3': 0.05677951548917238,    'l2_3': 7.714986268695782e-05,  'activation': 'relu', 'optimizer': 'AdamW', 'lr_initial': 0.006869840110235172, 'reduce_lr_factor': 0.2, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.028747844465778e-06,  'epochs': 51, 'batch_size': 479}.
# #Trial 99 finished with value:  0.9911666512489319 and parameters: {'num_filters1': 222, 'num_filters2': 220, 'num_filters3': 202, 'PCA_components': 367, 'l1_1': 4.875844948451946e-06,  'l2_1': 0.00014963179483695857, 'l1_2': 2.3622464527905993e-06, 'l2_2': 0.0007506481920246097,  'l1_3': 0.030678868126237845,   'l2_3': 0.00016514043221099142, 'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.00653453290497133,  'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.8941869280860333e-06, 'epochs': 54, 'batch_size': 331}.
# #Trial 108 finished with value: 0.9915000200271606 and parameters: {'num_filters1': 205, 'num_filters2': 206, 'num_filters3': 220, 'PCA_components': 370, 'l1_1': 9.95600828812184e-06,   'l2_1': 4.680137051239973e-05,  'l1_2': 4.9076000996726795e-06, 'l2_2': 8.638060092445147e-05,  'l1_3': 0.0361709248993415,     'l2_3': 0.0007397174884371887,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006269166650314777, 'reduce_lr_factor': 0.2, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 5.203896150606987e-06,  'epochs': 62, 'batch_size': 239}.
# #Trial 121 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 189, 'num_filters2': 200, 'num_filters3': 220, 'PCA_components': 387, 'l1_1': 8.32443244291399e-05,   'l2_1': 5.963850113018811e-05,  'l1_2': 4.8017415472934674e-06, 'l2_2': 3.9423405128005956e-05, 'l1_3': 0.00047971791859799064, 'l2_3': 0.0014900662405900767,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006533479568732247, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 6, 'reduce_lr_min_lr': 7.315164005922789e-06,  'epochs': 58, 'batch_size': 249}.
# #Trial 134 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 180, 'num_filters2': 196, 'num_filters3': 234, 'PCA_components': 390, 'l1_1': 4.148854564296224e-05,  'l2_1': 2.3209137723337733e-05, 'l1_2': 0.0009209306936723504,  'l2_2': 1.9816087702628798e-05, 'l1_3': 0.0008993453039409338,  'l2_3': 0.002218879345590759,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007404316715937198, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 3.862546224445297e-07,  'epochs': 61, 'batch_size': 368}.
# #Trial 151 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 201, 'num_filters2': 189, 'num_filters3': 229, 'PCA_components': 403, 'l1_1': 6.15892270118e-05,      'l2_1': 1.6056284141459273e-05, 'l1_2': 0.0009167391639733724,  'l2_2': 1.4409315946656477e-05, 'l1_3': 0.0008899963895277467,  'l2_3': 0.005080838638011425,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007378956521980373, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 1.9759552872681682e-07, 'epochs': 63, 'batch_size': 416}.
# #Trial 153 finished with value: 0.9905833601951599 and parameters: {'num_filters1': 158, 'num_filters2': 190, 'num_filters3': 229, 'PCA_components': 405, 'l1_1': 3.317865892202326e-05,  'l2_1': 1.715760099034611e-05,  'l1_2': 0.00028475050803045533, 'l2_2': 7.144046413508896e-06,  'l1_3': 0.0010057548651594868,  'l2_3': 0.001544820248650564,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.007667044780464368, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 1.6284942102179344e-07, 'epochs': 64, 'batch_size': 420}.
# #Trial 189 finished with value: 0.9904166460037231 and parameters: {'num_filters1': 187, 'num_filters2': 194, 'num_filters3': 191, 'PCA_components': 381, 'l1_1': 0.0001222508587314991,  'l2_1': 5.2900939483419044e-05, 'l1_2': 4.050810455502403e-05,  'l2_2': 0.00017281194315162931, 'l1_3': 0.0014341620209873955,  'l2_3': 0.0025511470455798512,  'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006996548564114207, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 6, 'reduce_lr_min_lr': 9.663271049317225e-07,  'epochs': 60, 'batch_size': 219}.
# #Trial 205 finished with value: 0.9904999732971191 and parameters: {'num_filters1': 210, 'num_filters2': 189, 'num_filters3': 198, 'PCA_components': 483, 'l1_1': 8.253178154361272e-05,  'l2_1': 1.7866032331171518e-05, 'l1_2': 0.0008019764970918962,  'l2_2': 1.0316537375038554e-05, 'l1_3': 0.000838618983241391,   'l2_3': 0.008249547665798275,   'activation': 'mish', 'optimizer': 'AdamW', 'lr_initial': 0.006570979651986537, 'reduce_lr_factor': 0.1, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 2.6114120319881016e-07, 'epochs': 63, 'batch_size': 560}.


#### V2 dynamic PCA

In [ ]:
# import tensorflow as tf
# import keras
# from keras import layers
# import tensorflow_probability as tfp
# import optuna
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 

# X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
# X_train_image = X_train_image.astype('float32')
# X_val_image = X_val_image.astype('float32')
# X_train_image = np.expand_dims(X_train_image, axis=-1)  
# X_val_image = np.expand_dims(X_val_image, axis=-1) 
 
# def objective(trial):
#     # Przestrzeń hiperparametrów
#     num_filters1 = trial.suggest_int("num_filters1", 180, 230)    # Pierwsza warstwa Conv2d
#     num_filters2 = trial.suggest_int("num_filters2", 180, 230)    # Druga warstwa Conv2d
#     num_filters3 = trial.suggest_int("num_filters3", 180, 260)    # Trzecia warstwa Conv2d
#     n_components = trial.suggest_int("PCA_components", 350, 420)  # PCA
    
#     l1_1=trial.suggest_float('l1_1', 1e-6, 5e-3, log=True)
#     l2_1=trial.suggest_float('l2_1', 5e-5, 5e-3, log=True)
    
#     l1_2=trial.suggest_float('l1_2', 1e-6, 1e-3, log=True)
#     l2_2=trial.suggest_float('l2_2', 1e-6, 1e-5, log=True)

#     l1_3=trial.suggest_float('l1_3', 1e-6, 1e-1, log=True)
#     l2_3=trial.suggest_float('l2_3', 1e-6, 1e-1, log=True)
    
#     activation_functions = ['gelu', 'leaky_relu', 'silu', 'mish']
#     activation = trial.suggest_categorical('activation', activation_functions)
    
    
#     class DynamicPCA(keras.layers.Layer):
#         def __init__(self, n_components, layer, update_freq=2, threshold=0.5, **kwargs):  
#             super(DynamicPCA, self).__init__(**kwargs)
#             self.n_components = n_components
#             self.threshold = threshold
#             self.layer = layer  
#             self.update_freq = update_freq
#             self.val_loss = None
#             self.eigenvectors = None
#             self.epoch_count = 0

#         def build(self, input_shape):
#             self.eigenvectors = self.add_weight(
#                 shape=(input_shape[-1], self.n_components),
#                 initializer="zeros",
#                 trainable=True,
#             )

#         def call(self, inputs):
#             self.epoch_count += 1
#             if (self.val_loss is not None and 
#                 self.val_loss < self.threshold and 
#                 self.epoch_count % self.update_freq == 0):
#                 print("DynamicPCA: Aktualizacja wektorów własnych!")
#                 print(f"val_loss: {self.val_loss}, threshold: {self.threshold}, epoch_count: {self.epoch_count}, update_freq: {self.update_freq}")
                
#                 # Pobierz wagi bezpośrednio z warstwy
#                 weights = self.layer.get_weights()  
#                 flat_weights = tf.concat([tf.reshape(w, [-1]) for w in weights], axis=0)
#                 # Normalizacja wag
#                 flat_weights = tf.math.l2_normalize(flat_weights)
#                 # Obliczanie nowych wektorów własnych
#                 new_eigenvectors = self.calculate_pca(flat_weights)
#                 # Aktualizacja wag (wektorów własnych)
#                 self.eigenvectors.assign(new_eigenvectors)

#             # Mnożenie przez wektory własne
#             output = tf.matmul(inputs, self.eigenvectors)
#             return output

#         def calculate_pca(self, inputs):
#             print(flat_weights.shape)
#             # Obliczanie macierzy kowariancji
#             covariance_matrix = tf.linalg.cov(inputs)  
#             # Wektory i wartości własne
#             eigenvalues, eigenvectors = tf.linalg.eigh(covariance_matrix)
#             sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING')
#             eigenvectors = tf.gather(eigenvectors, sorted_indices, axis=1)
#             return eigenvectors[:, :self.n_components]

#     class MyCallback(keras.callbacks.Callback):
#         def on_epoch_end(self, epoch, logs=None):
#             pca_layer = self.model.get_layer('dynamic_pca')
#             pca_layer.val_loss = logs.get('val_loss')
#             print(logs.get('val_loss'))

#     # Definiowanie wejścia
#     image_input = keras.Input(shape=X_train_image.shape[1:])  

#     # Warstwy konwolucyjne
#     conv1 = keras.layers.Conv2D(num_filters1, kernel_size=(2, 2), activation=activation)(image_input)
#     pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = keras.layers.Conv2D(num_filters2, kernel_size=(2, 2), activation=activation)(pool1)
#     pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = keras.layers.Conv2D(num_filters3, kernel_size=(2, 2), activation=activation)(pool2)
#     pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
#     flat_cnn = keras.layers.Flatten()(pool3)

#     # Dense 1
#     dense = keras.layers.Dense(units=trial.suggest_int('dense_units_1', 60, 240), activation=activation)(flat_cnn) 
#     dense = keras.layers.Dropout(rate=trial.suggest_float('dropout_rate_1', 0.01, 0.75))(dense)

#     # Warstwa DynamicPCA 
#     pca_output = DynamicPCA(n_components=n_components, layer=conv3, name='dynamic_pca', threshold=1.25)(flat_cnn)  
    
#     merged = keras.layers.Concatenate()([dense, pca_output])

#     dense2 = keras.layers.Dense(units=trial.suggest_int('dense_units_2', 60, 240), activation=activation)(merged)  
#     dense2 = keras.layers.Dropout(rate=trial.suggest_float('dropout_rate_2', 0.01, 0.75))(dense2)

#     # Output (klasyfikacja)
#     outputs = keras.layers.Dense(10, activation='softmax')(dense2)

#     # Utworzenie modelu
#     model = keras.Model(inputs=image_input, outputs=outputs)

#     # Utworzenie modelu
#     model = keras.Model(inputs=image_input, outputs=outputs)
#     #model.summary()
#     tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    

#     # Optymalizator
#     optimizer_name = trial.suggest_categorical('optimizer', ['AdamW'])
#     optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
#     # Początkową wartość learning rate
#     lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
#     optimizer = optimizer_class(learning_rate=lr_initial)  

#     ## Kompilacja modelu
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     # Early Stopping
#     early_stopping = EarlyStopping(monitor='categorical_accuracy', patience=10, start_from_epoch=8, min_delta=0.0001)
    
#     # ReduceLROnPlateau
#     reduce_lr = ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.5, step=0.1),
#         patience=trial.suggest_int("reduce_lr_patience", 4, 8),
#         min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-5, log=True)
#     )
    
#     # Trenowanie i ocena modelu
#     history = model.fit(X_train_image, y_train_image,
#                         epochs=trial.suggest_int('epochs', 20, 80),
#                         batch_size=trial.suggest_int('batch_size', 164, 768),
#                         validation_data=(X_val_image, y_val_image),
#                         validation_freq=1,
#                         callbacks=[early_stopping, reduce_lr, MyCallback()]
#                         )
    
#     accuracy = history.history['val_categorical_accuracy'][-1]

#     return accuracy


# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# study.optimize(objective, n_trials=600, n_jobs=1, gc_after_trial=True) 

# # Najlepsze parametry i wynik
# print("Najlepsze parametry:", study.best_params)
# print("Najlepsza dokładność:", study.best_value)



## Model bazujący na dwóch potokach danych. Surowych przechodzących przez Conv2d, oraz LSTM dla danych z HOG i Gabor. Wykorzystanie mechanizmu atencji. 

In [ ]:
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import Constraint

from tensorflow.keras.layers import ( Input, LSTM, GlobalAveragePooling1D, Permute, PReLU, Flatten, Reshape, Conv2D, MaxPooling2D, Concatenate, Attention, Dense,
                                     LayerNormalization, ActivityRegularization, Dropout, GlobalAveragePooling2D, AveragePooling2D, AveragePooling1D, GlobalAveragePooling1D, 
                                     GlobalMaxPool1D, GlobalMaxPool2D, MaxPool2D, SeparableConv1D, RandomRotation, BatchNormalization
                                    )
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gc
from tensorflow.keras.callbacks import TensorBoard

X_train_reduced = np.load('X_train_reduced.npy')
X_val_reduced = np.load('X_val_reduced.npy')
y_train_filters = np.load('y_train_filters.npy')
y_val_filters = np.load('y_val_filters.npy')

X_train_reduced = X_train_reduced.reshape(X_train_reduced.shape[0], X_train_reduced.shape[1], 1)
X_val_reduced = X_val_reduced.reshape(X_val_reduced.shape[0], X_val_reduced.shape[1], 1)

X_train_image, X_val_image, y_train_image, y_val_image = train_test_split(X_train, y_train_ENC, test_size=0.2, random_state=69)
X_train_image = X_train_image.astype('float32')
X_val_image = X_val_image.astype('float32')
X_train_image = np.expand_dims(X_train_image, axis=-1)  
X_val_image = np.expand_dims(X_val_image, axis=-1) 

class SpectralNorm(Constraint):
    """
    Spectral Normalization constraint.

    Args:
        iteration: Number of iterations for power iteration.
    """
    def __init__(self, iteration=1):
        super(SpectralNorm, self).__init__()
        self.iteration = iteration
        self.u = None

    def __call__(self, w):
        w_shape = w.shape.as_list()
        w = tf.reshape(w, [-1, w_shape[-1]])

        if self.u is None:
            self.u = tf.Variable(
                initial_value=tf.keras.initializers.RandomNormal()(shape=[1, w_shape[-1]]),
                trainable=False
            )

        if self.iteration == 1:
            v_hat = tf.einsum('ij,kj->ik', self.u, w) 
            v_hat = v_hat / tf.linalg.norm(v_hat, ord=2, axis=-1, keepdims=True)
            u_hat = tf.einsum('ij,jk->ik', v_hat, w)
            u_hat = u_hat / tf.linalg.norm(u_hat, ord=2, axis=-1, keepdims=True)
        else:
            def body(i, u_hat):
                v_hat = tf.einsum('ij,kj->ik', u_hat, w)
                v_hat = v_hat / tf.linalg.norm(v_hat, ord=2, axis=-1, keepdims=True)
                u_hat = tf.einsum('ij,jk->ik', v_hat, w)
                u_hat = u_hat / tf.linalg.norm(u_hat, ord=2, axis=-1, keepdims=True)
                return i + 1, u_hat

            _, u_hat = tf.while_loop(
                lambda i, _: i < self.iteration,
                body,
                loop_vars=[0, self.u]
            )

        self.u.assign(u_hat)  # Aktualizacja self.u po pętli

        w_norm = w / tf.linalg.norm(w, ord=2, axis=0, keepdims=True)
        w_norm = tf.reshape(w_norm, w_shape)

        return w_norm
    
def create_sepconv1d_layer(units, activation, l1, l2):
    return SeparableConv1D(
        units,
        kernel_size=1,
        padding='same',
        activation=activation,
        depthwise_constraint=SpectralNorm,
        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    )

def create_conv2d_layer(filters, activation, l1, l2):
    return Conv2D(
        filters,
        kernel_size=(2, 2),
        activation=activation,
        kernel_constraint=SpectralNorm,
        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)
    )    
    
def objective(trial):
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    # Dane wejściowe
    image_input = Input(shape=X_train_image.shape[1:])
    flattened_input = Input(shape=X_train_reduced.shape[1:])

    ### Hiperparamatry 
    activation_functions = ['gelu', 'leaky_relu', 'silu', 'mish']
    activation = trial.suggest_categorical('activation', activation_functions)
        
    filters1_1D=trial.suggest_int('Sep-Conv2d filters1', 32, 384)  # Pierwsza warstwa SeparableConv1D
    filters2_1D=trial.suggest_int('Sep-Conv2d filters2', 32, 384)  # Druga warstwa SeparableConv1D
    filters3_1D=trial.suggest_int('Sep-Conv2d filters3', 32, 384)  # Trzecia warstwa SeparableConv1D
    
    l1_1_1D=trial.suggest_float('l1_1_1D', 1e-6, 1e-1, log=True)
    l2_1_1D=trial.suggest_float('l2_1_1D', 1e-6, 1e-1, log=True)
    l1_2_1D=trial.suggest_float('l1_2_1D', 1e-6, 1e-1, log=True)
    l2_2_1D=trial.suggest_float('l2_2_1D', 1e-6, 1e-1, log=True)
    l1_3_1D=trial.suggest_float('l1_3_1D', 1e-6, 1e-1, log=True)
    l2_3_1D=trial.suggest_float('l2_3_1D', 1e-6, 1e-1, log=True)
       
    filters1_2D = trial.suggest_int("Conv2d filters1", 32, 384)    # Pierwsza warstwa Conv2d
    filters2_2D = trial.suggest_int("Conv2d filters2", 32, 384)    # Druga warstwa Conv2d
    filters3_2D = trial.suggest_int("Conv2d filters3", 32, 384)    # Trzecia warstwa Conv2d

    l1_1_2D=trial.suggest_float('l1_1_2D', 1e-6, 1e-1, log=True)
    l2_1_2D=trial.suggest_float('l2_1_2D', 1e-6, 1e-1, log=True)
    l1_2_2D=trial.suggest_float('l1_2_2D', 1e-6, 1e-1, log=True)
    l2_2_2D=trial.suggest_float('l2_2_2D', 1e-6, 1e-1, log=True)
    l1_3_2D=trial.suggest_float('l1_3_2D', 1e-6, 1e-1, log=True)
    l2_3_2D=trial.suggest_float('l2_3_2D', 1e-6, 1e-1, log=True)

    # Gałąź SeparableConv1D
    Sep_Conv1D_1 = create_sepconv1d_layer(filters1_1D, activation, l1_1_1D, l2_1_1D)(flattened_input)
    Sep_Conv1D_2 = create_sepconv1d_layer(filters2_1D, activation, l1_2_1D, l2_2_1D)(Sep_Conv1D_1)
    Sep_Conv1D_3 = create_sepconv1d_layer(filters3_1D, activation, l1_3_1D, l2_3_1D)(Sep_Conv1D_2)
    # Pooling 
    Sep_pool_Conv1D = GlobalAveragePooling1D()(Sep_Conv1D_3)    

    # Gałąź Conv2D
        ## Pierwsza warstwa
    conv1 = create_conv2d_layer(filters1_2D, activation, l1_1_2D, l2_1_2D)(image_input)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        ## Druga warstwazz
    conv2 = create_conv2d_layer(filters2_2D, activation, l1_2_2D, l2_2_2D)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        ## Trzecia warstwa
    conv3 = create_conv2d_layer(filters3_2D, activation, l1_3_2D, l2_3_2D)(pool2)
    pool3 = MaxPool2D(pool_size=(3, 3))(conv3)  
    
    flat_cnn = Flatten()(pool3)

    #Dense gałąź SeparableConv1D
    dense1 = Dense(units=trial.suggest_int('dense SeparableConv1D', 32, 384), activation=activation)(Sep_pool_Conv1D)
    dense1 = Dropout(rate=trial.suggest_float('dropout_rate_1', 0.0, 0.75))(dense1)  

    #Dense gałąź Conv2D    
    dense2 = Dense(units=trial.suggest_int('dense Conv2D', 32, 384), activation=activation)(flat_cnn)
    dense2 = Dropout(rate=trial.suggest_float('dropout_rate_2', 0.0, 0.75))(dense2)
    
    # Połączenie gałęzi
    merged = Concatenate()([dense1, dense2])

    merged = Reshape((merged.shape[1], 1))(merged)

    # Attention Layer
    attention_output = Attention()([merged, merged])
    attention_output = Reshape((attention_output.shape[1],))(attention_output)
    
    #Dense
    dense3 = Dense(units=trial.suggest_int('dense przed Output', 36, 384), activation=activation)(attention_output)
    dense3 = Dropout(rate=trial.suggest_float('dropout przed Output', 0.0, 0.75))(dense3)

    #Output
    output = Dense(10, activation='softmax')(dense3)
    model = Model(inputs=[image_input, flattened_input], outputs=output)
    
    #tf.keras.utils.plot_model(model, show_shapes=True, to_file='model.png')
    #model.summary()
    
    # ReduceLROnPlateau
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=trial.suggest_float("reduce_lr_factor", 0.1, 0.6, step=0.1),
        patience=trial.suggest_int("reduce_lr_patience", 2, 14),
        min_lr=trial.suggest_float("reduce_lr_min_lr", 1e-7, 1e-4, log=True)
    )

    # Optymalizator
    optimizer_name = trial.suggest_categorical('optimizer', ['AdamW', 'Lion'])
    optimizer_class = getattr(tf.keras.optimizers, optimizer_name)
    
    # Początkową wartość learning rate
    lr_initial = trial.suggest_float("lr_initial", 5e-3, 1e-1, log=True)
    optimizer = optimizer_class(learning_rate=lr_initial)  

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=6, start_from_epoch=12, min_delta=0.0001)
    #log_dir = "logs/"  # Katalog, w którym będą zapisywane dane TensorBoard
    #tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1 )

    # Trenowanie modelu
    history = model.fit(
        [X_train_image, X_train_reduced], 
        y_train_filters,
        epochs=trial.suggest_int('epochs', 20, 90),
        batch_size=trial.suggest_int('batch_size', 164, 768),
        validation_data=([X_val_image, X_val_reduced], y_val_filters),
        verbose=1,
        callbacks=[early_stopping, reduce_lr],# , tensorboard
    )

    best_val_accuracy = history.history['val_categorical_accuracy'][-1]

    # Ewaluacja modelu na danych testowych
    #loss, accuracy = model.evaluate([X_test, X_test], y_test_ENC, verbose=1)
    #print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

    del model
    return best_val_accuracy

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=200, n_jobs=1, gc_after_trial=True) ## Czy keras działa na wszystkich wątkach, ale optuna na pojedynczym. 


# Najlepsze parametry i wynik
print("Najlepsze parametry:", study.best_params)
print("Najlepsza dokładność:", study.best_value)

#[I 2024-10-25 02:17:14,226] Trial 55 finished with value:  0.9590833187103271 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 23,  'Sep-Conv2d filters2': 58,  'Sep-Conv2d filters3': 85, 'l1_1_1D': 0.0002263330199021373,   'l2_1_1D': 4.2547366301065874e-06,  'l1_2_1D': 1.8831463765106328e-05,  'l2_2_1D': 2.6849906489521574e-06, 'l1_3_1D': 5.376750360962702e-06, 'l2_3_1D': 4.5507289814394546e-05, 'Conv2d filters1': 27,  'Conv2d filters2': 75, 'Conv2d filters3': 66,  'l1_1_2D': 0.03570924628671723,    'l2_1_2D': 0.00013702268815391088,'l1_2_2D': 0.004180188847206343,   'l2_2_2D': 0.015697748113667306,   'l1_3_2D': 1.1093102758419453e-06, 'l2_3_2D': 2.3039556898400452e-05, 'dense SeparableConv1D': 113, 'dropout_rate_1': 0.25988069757258775, 'dense Conv2D': 55,  'dropout_rate_2': 0.1729409318039701,   'dense przed Output': 213, 'dropout przed Output': 0.692278722477647,    'reduce_lr_factor': 0.1, 'reduce_lr_patience': 8, 'reduce_lr_min_lr': 1.2382721420683838e-05, 'optimizer': 'AdamW', 'lr_initial': 0.007665780845145814, 'epochs': 81, 'batch_size': 173}. Best is trial 55 with value: 0.9590833187103271.
#[I 2024-10-25 05:23:25,094] Trial 82 finished with value:  0.9622499942779541 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 22,  'Sep-Conv2d filters2': 28,  'Sep-Conv2d filters3': 100, 'l1_1_1D': 6.186293277794836e-05,  'l2_1_1D': 0.0003913398305461908,   'l1_2_1D': 1.5927923474696962e-06,  'l2_2_1D': 7.009282960520817e-06,  'l1_3_1D': 7.676137877709806e-06, 'l2_3_1D': 0.00022663346915615475, 'Conv2d filters1': 24,  'Conv2d filters2': 74, 'Conv2d filters3': 92,  'l1_1_2D': 0.0993169614612255,     'l2_1_2D': 0.0006883072766765725, 'l1_2_2D': 0.0005616491418975441,  'l2_2_2D': 1.0689411586574192e-05, 'l1_3_2D': 4.1278686535916746e-05, 'l2_3_2D': 4.837146281888261e-06,  'dense SeparableConv1D': 155, 'dropout_rate_1': 0.0509776183623675,  'dense Conv2D': 29,  'dropout_rate_2': 0.024717745882008837, 'dense przed Output': 178, 'dropout przed Output': 0.7445169215013963, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 4, 'reduce_lr_min_lr': 6.725637729646196e-06,  'optimizer': 'AdamW', 'lr_initial': 0.01728017154953551,  'epochs': 85, 'batch_size': 191}. Best is trial 82 with value: 0.9622499942779541.
#[I 2024-10-25 15:22:24,767] Trial 142 finished with value: 0.9635000228881836 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 22,  'Sep-Conv2d filters2': 28,  'Sep-Conv2d filters3': 107, 'l1_1_1D': 6.120269265952697e-05,  'l2_1_1D': 7.956576920829512e-06,   'l1_2_1D': 1.2494724596667829e-06,  'l2_2_1D': 7.6581362167544e-06,    'l1_3_1D': 0.0039961697402018225, 'l2_3_1D': 0.00021250335195961356, 'Conv2d filters1': 28,  'Conv2d filters2': 72, 'Conv2d filters3': 99 , 'l1_1_2D': 0.07544375319709569,    'l2_1_2D': 0.000936229620640654,  'l1_2_2D': 0.001250431002984649,   'l2_2_2D': 0.01324640444949233,    'l1_3_2D': 0.0012955495310992596,  'l2_3_2D': 6.618279886291623e-06,  'dense SeparableConv1D': 151, 'dropout_rate_1': 0.08650725184529029, 'dense Conv2D': 29,  'dropout_rate_2': 0.0500476697674726,   'dense przed Output': 205, 'dropout przed Output': 0.7360088478595872,   'reduce_lr_factor': 0.4, 'reduce_lr_patience': 7, 'reduce_lr_min_lr': 9.207463977521836e-07,  'optimizer': 'AdamW', 'lr_initial': 0.01209682198565378,  'epochs': 88, 'batch_size': 228}. Best is trial 142 with value: 0.9635000228881836.
#[I 2024-11-01 00:07:23,390] Trial 24 finished with value:  0.9449999928474426 and parameters: {'activation': 'mish',        'Sep-Conv2d filters1': 123, 'Sep-Conv2d filters2': 245, 'Sep-Conv2d filters3': 289, 'l1_1_1D': 0.08989218874411514,    'l2_1_1D': 1.1472596089503972e-06,  'l1_2_1D': 0.0026057385621594867,   'l2_2_1D': 0.00018978883951458408, 'l1_3_1D': 1.191033349842715e-06, 'l2_3_1D': 0.00014292949482563498, 'Conv2d filters1': 308, 'Conv2d filters2': 81, 'Conv2d filters3': 326, 'l1_1_2D': 2.9840369417992957e-06, 'l2_1_2D': 0.0014037071972283193, 'l1_2_2D': 0.0002640492511862146,  'l2_2_2D': 0.08499221386872775,    'l1_3_2D': 0.020374031922578733,   'l2_3_2D': 0.00010028036351417339, 'dense SeparableConv1D': 379, 'dropout_rate_1': 0.0544686509758308,  'dense Conv2D': 138, 'dropout_rate_2': 0.09593865740145555,  'dense przed Output': 384, 'dropout przed Output': 0.6382663359384837, 'reduce_lr_factor': 0.3, 'reduce_lr_patience': 4, 'reduce_lr_min_lr': 7.741233462096205e-06,  'optimizer': 'AdamW', 'lr_initial': 0.006707953904631179, 'epochs': 65, 'batch_size': 257}. Best is trial 24 with value: 0.9449999928474426.
#[I 2024-11-05 15:57:39,996] Trial 101 finished with value: 0.9495833516120911 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 148, 'Sep-Conv2d filters2': 32,  'Sep-Conv2d filters3': 63, 'l1_1_1D': 0.0004897725865476358,   'l2_1_1D': 0.0003049226492905848,   'l1_2_1D': 5.00894966144464e-06,    'l2_2_1D': 0.00014051697363473006, 'l1_3_1D': 0.051135669099956614,  'l2_3_1D': 0.0009950758155242705,  'Conv2d filters1': 52,  'Conv2d filters2': 58, 'Conv2d filters3': 249, 'l1_1_2D': 0.014787666006730659,   'l2_1_2D': 0.0005948328558539133, 'l1_2_2D': 5.343054689024963e-06,  'l2_2_2D': 4.5277461875969285e-06, 'l1_3_2D': 1.9548002422161054e-06, 'l2_3_2D': 0.0009410443070360079,  'dense SeparableConv1D': 319, 'dropout_rate_1': 0.2533943711642646,  'dense Conv2D': 311, 'dropout_rate_2': 0.10208101037728201,  'dense przed Output': 90,  'dropout przed Output': 0.481006546242034,   'reduce_lr_factor': 0.5, 'reduce_lr_patience': 2, 'reduce_lr_min_lr': 5.559247664816983e-05,  'optimizer': 'AdamW', 'lr_initial': 0.00964928371074687,  'epochs': 66, 'batch_size': 423}. Best is trial 101 with value: 0.9495833516120911.
#[I 2024-11-06 02:29:49,053] Trial 121 finished with value: 0.9535833597183228 and parameters: {'activation': 'leaky_relu',  'Sep-Conv2d filters1': 119, 'Sep-Conv2d filters2': 49,  'Sep-Conv2d filters3': 158, 'l1_1_1D': 0.00023280964689116444, 'l2_1_1D': 0.0008751788463496611,   'l1_2_1D': 0.04610519254480619,     'l2_2_1D': 2.4445376899197913e-06, 'l1_3_1D': 0.06955230495640688,   'l2_3_1D': 0.004678025581652909,   'Conv2d filters1': 37,  'Conv2d filters2': 54, 'Conv2d filters3': 253, 'l1_1_2D': 0.016409226434426858,  'l2_1_2D': 0.00015292748828121323, 'l1_2_2D': 1.177904875672071e-05,  'l2_2_2D': 2.6668468645182726e-06, 'l1_3_2D': 6.7648129463989756e-06, 'l2_3_2D': 0.0007509210891800366,  'dense SeparableConv1D': 347, 'dropout_rate_1': 0.2855797128973145,  'dense Conv2D': 368, 'dropout_rate_2': 0.07539865516979521,  'dense przed Output': 73,  'dropout przed Output': 0.5059616088424707, 'reduce_lr_factor': 0.4, 'reduce_lr_patience': 2, 'reduce_lr_min_lr': 3.887676551167433e-05, 'optimizer': 'AdamW', 'lr_initial': 0.00852437932688485, 'epochs': 73, 'batch_size': 407}. Best is trial 121 with value: 0.9535833597183228.